# Part 2: Cleaning Data

Can we predict who will win an NBA game, given only information before the game has started?

## The Plan
- Part 1: Scraping NBA.com Data
- Part 2: Scraping team & player images
- Part 3: Get all Game-by-Game data
- Part 4: Initial Model(s)
- Part 5: Add player-level data
- Part 6: More Models
- Part 7: Conclusion

In [3]:
#Load
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import requests     # to get images
import shutil       # to save files locally
import datetime
from scipy.stats import norm
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
import os
from pathlib import Path
# NBA API
#from nba_api.stats.static import players
#from nba_api.stats.endpoints import commonplayerinfo
#from nba_api.stats.static import teams

# Part 2: Cleaning and combining data

## Team Advanced Game-by-Game Boxscores

First, I want to get all the data for each game in one dataframe. 
I will start with the advanced game statistics

In [ ]:
team_advbox_21 = pd.read_excel('team_boxes\\boxscores-advanced_SeasonType_Regular_20Season_Season_2021-22_SeasonYear_2021-22.xlsx')
team_advbox_21 = team_advbox_21[1:]
team_advbox_21['Season'] = '2021-2022'
team_advbox_21.head(3)

In [ ]:
# Add team-game id with Team & Date
team_advbox_21['team_date_id'] = team_advbox_21['Team'] + "_" + team_advbox_21['Game\xa0Date']
team_advbox_21.head(3)

In [ ]:
# Now for the normal box scores
team_box_21 = pd.read_excel('team_boxes\\boxscores-traditional_SeasonType_Regular_20Season_Season_2021-22_SeasonYear_2021-22.xlsx')
team_box_21 = team_box_21[1:]
team_box_21['Season'] = '2021-2022'
team_box_21.head(3)


In [ ]:
team_box_21['team_date_id'] = team_box_21['Team'] + "_" + team_box_21['Game\xa0Date']
team_box_21.head(3)

In [ ]:
team_box_21_combined = pd.merge(team_advbox_21, team_box_21, on= 'team_date_id')
team_box_21_combined.head(3)

### Team Box Score - Scoring

In [ ]:
# Scoring Box Score
team_scorebox_21 = pd.read_excel('team_boxes\\boxscores-scoring_SeasonType_Regular_20Season_Season_2021-22_SeasonYear_2021-22.xlsx')
team_scorebox_21 = team_scorebox_21[1:]
team_scorebox_21['Season'] = '2021-2022'
team_scorebox_21.head(3)

In [ ]:
team_scorebox_21 = team_scorebox_21.drop(columns= ['Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27'])

In [ ]:
team_scorebox_21.head(3)

In [ ]:
team_scorebox_21['team_date_id'] = team_scorebox_21['Team'] + "_" + team_scorebox_21['Game_date']

In [ ]:
team_scorebox_21

In [ ]:
team_box_21_combined = pd.merge(team_box_21_combined, team_scorebox_21, on= 'team_date_id')
team_box_21_combined.head(3)

### Team Box Score - Four Factors

In [ ]:
team_ff_box_21 = pd.read_excel('team_boxes\\boxscores-four-factors_SeasonType_Regular_20Season_Season_2021-22_SeasonYear_2021-22.xlsx')
team_ff_box_21 = team_ff_box_21[1:]
team_ff_box_21['Season'] = '2021-2022'
team_ff_box_21

In [ ]:
team_ff_box_21['team_date_id'] = team_ff_box_21['Team'] + "_" + team_ff_box_21['Game\xa0Date']

In [ ]:
team_box_21_combined = pd.merge(team_box_21_combined, team_ff_box_21, on= 'team_date_id')
team_box_21_combined.head()

### Team Box Score - Misc

In [ ]:
team_misc_box_21 = pd.read_excel('team_boxes\\boxscores-misc_SeasonType_Regular_20Season_Season_2021-22_SeasonYear_2021-22.xlsx')
team_misc_box_21 = team_misc_box_21[1:]
team_misc_box_21['Season'] = '2021-2022'
team_misc_box_21

In [ ]:
team_misc_box_21['team_date_id'] = team_misc_box_21['Team'] + "_" + team_misc_box_21['Game\xa0Date']

In [ ]:
team_box_21_combined = pd.merge(team_box_21_combined, team_misc_box_21, on= 'team_date_id')
team_box_21_combined.head()

In [ ]:
team_box_21_combined.shape

In [ ]:
# make folder for data outputs after python work
#os.mkdir('data_outputs')

In [ ]:
# Save the combined team box score for 2021
team_box_21_combined.to_excel('data_outputs\\Combined_Team_Boxes_2021-22.xlsx')

### Team Advanced ALL YEARS Join

In [ ]:
for team in teams:
    tab_21 = pd.read_excel('team_boxes\\boxscores-advanced_SeasonType_Regular_20Season_Season_2021-22_SeasonYear_2021-22.xlsx')


In [ ]:
tab_21 = pd.read_excel('team_boxes\\boxscores-advanced_SeasonType_Regular_20Season_Season_2021-22_SeasonYear_2021-22.xlsx')
tab_20 = pd.read_excel('team_boxes\\boxscores-advanced_SeasonType_Regular_20Season_Season_2020-21_SeasonYear_2020-21.xlsx')
tab_19 = pd.read_excel('team_boxes\\boxscores-advanced_SeasonType_Regular_20Season_Season_2019-20_SeasonYear_2019-20.xlsx')
tab_18 = pd.read_excel('team_boxes\\boxscores-advanced_SeasonType_Regular_20Season_Season_2018-19_SeasonYear_2018-19.xlsx')
tab_17 = pd.read_excel('team_boxes\\boxscores-advanced_SeasonType_Regular_20Season_Season_2017-18_SeasonYear_2017-18.xlsx')

In [ ]:
tab_21.head(3)

In [ ]:
tabs = [tab_21, tab_20, tab_19, tab_18, tab_17 ]

In [ ]:
tabzz = []

for t in tabs:
    t = t[1:]
    tabzz.append(t)

tabzz = pd.concat(tabzz)

tabzz

In [ ]:
advanced_boxes_5yrs = tabzz

In [ ]:
advanced_boxes_5yrs['win'] = np.where(advanced_boxes_5yrs['W/L'] == "W", 1, 0)

In [ ]:
list(advanced_boxes_5yrs.columns)

In [ ]:
advanced_boxes_5yrs.rename(columns = {'Unnamed: 0' : 'indexy'}, inplace = True)

In [ ]:
ab5 = advanced_boxes_5yrs

In [ ]:
ab5.to_excel('data_outputs\\advancedboxes_5years.xlsx')


In [ ]:
advanced_boxes_5yrs

## Pre-EDA (Exploratory Data Analysis)

This gives us 101 variables per game. Let's see which are the most correlative to wins?

In [ ]:
team_box_21_combined = pd.read_excel('data_outputs\\Combined_Team_Boxes_2021-22.xlsx')

In [ ]:
#Adding column for win/loss equalling 1/0
team_box_21_combined['wins'] = np.where(team_box_21_combined['W/L_x'] == 'L', 0, 1)
team_box_21_combined.head()

In [ ]:
# Check out most correlated variables in the dataset
data_pred = team_box_21_combined
df=data_pred.corr().abs().stack().reset_index().sort_values(0, ascending=False)
df['pairs'] = list(zip(df.level_0, df.level_1))
df.set_index(['pairs'], inplace = True)
df.drop(columns=['level_1', 'level_0'], inplace = True)
df.columns = ['r']
df.drop_duplicates(inplace=True)
df[(df.r>.5) & (df.r <.99)].head(25)
df.head(10)

In [ ]:
# Only the win correlations
cors = team_box_21_combined.corr()
cors = cors['wins'].sort_values(ascending=False)
cors = pd.DataFrame(cors)
cors.head()

In [ ]:
df=cors.abs().reset_index().sort_values('wins', ascending = False)
df.drop_duplicates(inplace=True)

In [ ]:
df.head(5)

In [ ]:
team_box_21_combined['team_date_id']

## Adding Up-to-Date Season Averages

I performed this portion of the analysis in excel, as I am not sure how to do a averageifs function in python. *THIS MUST BE REMEDIED!*

In excel, I used the averageifs function, which appears like this:

=AVERAGEIFS(X$2:X$2161,$B$2:$B$2161, $V2, $A$2:$A$2161,"<= "& $A2)

In essense, I averaged each team's advanced box score variables, as long as the data used was known before the start of the game the teams are about to play.

In [ ]:
team_box_21_comb_SA = pd.read_excel('data_outputs\\Combined_Team_Boxes_2021-22_w_SeaAvg.xlsx')

In [ ]:
team_box_21_comb_SA

In [ ]:
#Adding column for win/loss equalling 1/0
team_box_21_comb_SA['wins'] = np.where(team_box_21_comb_SA['W/L_x'] == 'L', 0, 1)

In [ ]:
# Get rid of initial games - they are for collecting data. 
team_box_21_comb_SA = team_box_21_comb_SA[0:2100]
team_box_21_comb_SA.head()

## Starters and Injuries

For this, I want to be able to add players who are NOT playing in said games. This could possibly be done on a team-by-team basis. We can get this information from the player boxscores. 

First, I want to do this with a single team to see how it could get done. 

In [ ]:
# Add in the player data to find out who played in which games

play_1 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2021-22_sort_gdate_dir_-1_Month_1_SeasonType_Regular_20Season.xlsx')
play_2 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2021-22_sort_gdate_dir_-1_Month_2_SeasonType_Regular_20Season.xlsx')
play_3 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2021-22_sort_gdate_dir_-1_Month_3_SeasonType_Regular_20Season.xlsx')
play_4 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2021-22_sort_gdate_dir_-1_Month_4_SeasonType_Regular_20Season.xlsx')
play_5 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2021-22_sort_gdate_dir_-1_Month_5_SeasonType_Regular_20Season.xlsx')
play_6 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2021-22_sort_gdate_dir_-1_Month_6_SeasonType_Regular_20Season.xlsx')
play_7 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2021-22_sort_gdate_dir_-1_Month_7_SeasonType_Regular_20Season.xlsx')


In [ ]:
merged = play_1.append(play_2)
merged = merged[1:]
merged = merged.append(play_3)
merged = merged.append(play_4)
merged = merged.append(play_5)
merged = merged.append(play_6)
merged = merged.append(play_7)
merged.head(3)

### A) Add Team - Date - id

In [ ]:
merged['team_date_id'] = merged['Team'] + '_' + merged['Game\xa0Date']
merged.head(3)

In [ ]:
teams = merged.Team.unique()
teams

In [ ]:
player_traditional_box_scores_2021_22 = merged

### B) Save

In [ ]:
player_traditional_box_scores_2021_22.to_excel('data_outputs\\player_traditional_box_scores_2021_22.xlsx')

In [ ]:
#os.mkdir('team_players')

### C) Create sheets for each team, by year

In [ ]:
'''
for team in teams:
    player_boxes = merged.loc[merged['Team'] == team]
    player_boxes.to_excel('team_players\\'+ str(team) +'_player_boxes_21.xlsx')
    print (f' {team} done!')
'''    

### D) Create categorical variables for if a player on a team played in a game

In [ ]:
#os.mkdir('data_outputs_whoplayed_2')

In [ ]:
# Make new DF by team. 
for team in teams:
    cols = []
    player_boxes = merged.loc[merged['Team'] == team]
    player_namez = player_boxes['Player'].unique()        # Get unique player names
    for player in player_namez:                       # Add players as columns
        player_boxes[str(player) + '_played'] = np.where( player_boxes['Player'] == str(player), 1, 0)
        cols1 = str(player) + '_played'
        cols.append(cols1)
        player_boxes[str(player) + '_out'] = np.where(player_boxes[str(player) + '_played'] == '0', 1, 0)
        cols2 = str(player) + '_out'
        cols.append(cols2)
    grouped = player_boxes.groupby('team_date_id')[cols].sum()
    grouped.to_excel('data_outputs_whoplayed_2\\'+ str(team) +'_whoplayed_21.xlsx')
    print (f' {team} done!')


## Matchup data!
This seems very important, how different players do against different defenders, etc. 

The link is:

https://www.nba.com/game/dal-vs-gsw-0042100311/box-score?dir=D&sort=matchupMinutesSort&type=matchups

Looks like we'll need to use the NBA API to get the games. 

In [ ]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.static import teams
from nba_api.stats.library.parameters import SeasonAll

In [ ]:
from nba_api.stats.static import players
player_dict = players.get_players()
from nba_api.stats.static import teams 
teams = teams.get_teams()
teams

In [ ]:

from nba_api.stats.endpoints import playergamelog
import pandas as pd 
from nba_api.stats.library.parameters import SeasonAll


In [ ]:
#this time we convert it to a dataframe in the same line of code
ALL_games = leaguegamefinder.LeagueGameFinder().get_data_frames()[0]

In [ ]:
ALL_games['first_2']= ALL_games['GAME_ID'].astype(str).str[:2]
ALL_games


In [ ]:
nba_games = ALL_games.loc[ALL_games['first_2'] == '00']
nba_games

In [ ]:
# Add team date id

nba_games['team_date_id'] = nba_games['TEAM_ABBREVIATION'] + '_' + nba_games['GAME_DATE']

In [ ]:
nba_games

In [ ]:
nba_games['matchup2'] = nba_games['MATCHUP'].astype(str).str[:3] + '-vs-' + nba_games['MATCHUP'].astype(str).str[-3:]

In [ ]:
nba_games

In [ ]:
# Attempt to add the URL

nba_games['matchups_url'] = 'https://www.nba.com/game/'+nba_games['matchup2'] + '-' + nba_games['GAME_ID'] +'/box-score?dir=D&sort=matchupMinutesSort&type=matchups'

The link is:

https://www.nba.com/game/dal-vs-gsw-0042100311/box-score?dir=D&sort=matchupMinutesSort&type=matchups

In [ ]:
nba_games

In [ ]:
teams = gbg_advanced['Team'].unique()

## Who-Played by Team?

We could add them in by team, add in who DIDNT play, and then sum it all back together

In [ ]:
team_box_21_comb_SA.head() # Use this to connect to the WHO_PLAYED Dataframe

In [ ]:

for team in teams:
    data = team_box_21_comb_SA.loc[team_box_21_comb_SA['Team'] == team]
    data.to_excel('data_outputs\\' +str(team) + 'ad_box_21.xlsx')


In [ ]:
#os.mkdir('data_outputs_whoplayed')

#### For Loop to edit and combine 

In [ ]:
team_box_21_comb_SA.head(3)

In [ ]:
team_box_21_comb_SA['team_date_id'] = team_box_21_comb_SA['Team_x'] + "_" + team_box_21_comb_SA['Game\xa0Date_x']
team_box_21_comb_SA.head()

In [ ]:
teams = teams.astype(list)

In [ ]:
teams = teams[:-1]

In [ ]:
teams

In [ ]:
for team in teams:
    whop = 'data_outputs_whoplayed_2\\' + str(team) + '_whoplayed_21.xlsx'
    boxd = 'data_outputs\\' + str(team) + 'ad_box_21.xlsx' 
    whoplay = pd.read_excel(whop)
    boxdata = pd.read_excel(boxd)
    #merge
    data = pd.merge(boxdata, whoplay, on= 'team_date_id')
    data.to_excel('data_outputs_whoplayed_2\\' +str(team) + 'AdvBoxes_Whoplayed.xlsx')


In [ ]:
#os.mkdir('win_cors_by_team')

In [ ]:
for team in teams:
    combined = 'data_outputs_whoplayed\\' +str(team) + 'AdvBoxes_Whoplayed.xlsx'
    comb = pd.read_excel(combined)
    cor = comb.corr()
    winz = cor['wins'].sort_values(0, ascending= False).reset_index()
    winz.to_excel('win_cors_by_team\\' + str(team) + 'win_cor_21.xlsx')

## Who Played -- All Years Calculations

months_2019 = ['1', '2','3', '4', '5', '6']
months_2018 = ['1', '2','3', '4', '5', '6', '7']
months_2017 = ['1', '2','3', '4', '5', '6', '7']

In [ ]:
# 2020-21

play_1 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2020-21_sort_gdate_dir_-1_Month_3_SeasonType_Regular_20Season.xlsx')
play_2 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2020-21_sort_gdate_dir_-1_Month_4_SeasonType_Regular_20Season.xlsx')
play_3 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2020-21_sort_gdate_dir_-1_Month_5_SeasonType_Regular_20Season.xlsx')
play_4 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2020-21_sort_gdate_dir_-1_Month_6_SeasonType_Regular_20Season.xlsx')
play_5 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2020-21_sort_gdate_dir_-1_Month_7_SeasonType_Regular_20Season.xlsx')
play_6 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2020-21_sort_gdate_dir_-1_Month_8_SeasonType_Regular_20Season.xlsx')

merged = play_1.append(play_2)
merged = merged[1:]
merged = merged.append(play_3)
merged = merged.append(play_4)
merged = merged.append(play_5)
merged = merged.append(play_6)
merged.head(3)

player_traditional_box_scores_2020_21 = merged
player_traditional_box_scores_2020_21.to_excel('data_outputs\\player_traditional_box_scores_2020_21.xlsx')

In [ ]:
# 2019-20

play_1 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2019-20_sort_gdate_dir_-1_Month_1_SeasonType_Regular_20Season.xlsx')
play_2 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2019-20_sort_gdate_dir_-1_Month_2_SeasonType_Regular_20Season.xlsx')
play_3 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2019-20_sort_gdate_dir_-1_Month_3_SeasonType_Regular_20Season.xlsx')
play_4 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2019-20_sort_gdate_dir_-1_Month_4_SeasonType_Regular_20Season.xlsx')
play_5 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2019-20_sort_gdate_dir_-1_Month_5_SeasonType_Regular_20Season.xlsx')
play_6 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2019-20_sort_gdate_dir_-1_Month_6_SeasonType_Regular_20Season.xlsx')

merged = play_1.append(play_2)
merged = merged[1:]
merged = merged.append(play_3)
merged = merged.append(play_4)
merged = merged.append(play_5)
merged = merged.append(play_6)

player_traditional_box_scores_2019_20 = merged
player_traditional_box_scores_2019_20.to_excel('data_outputs\\player_traditional_box_scores_2019_20.xlsx')

In [ ]:
# 2018-19

play_1 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2018-19_sort_gdate_dir_-1_Month_1_SeasonType_Regular_20Season.xlsx')
play_2 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2018-19_sort_gdate_dir_-1_Month_2_SeasonType_Regular_20Season.xlsx')
play_3 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2018-19_sort_gdate_dir_-1_Month_3_SeasonType_Regular_20Season.xlsx')
play_4 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2018-19_sort_gdate_dir_-1_Month_4_SeasonType_Regular_20Season.xlsx')
play_5 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2018-19_sort_gdate_dir_-1_Month_5_SeasonType_Regular_20Season.xlsx')
play_6 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2018-19_sort_gdate_dir_-1_Month_6_SeasonType_Regular_20Season.xlsx')
play_7 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2018-19_sort_gdate_dir_-1_Month_7_SeasonType_Regular_20Season.xlsx')

merged = play_1.append(play_2)
merged = merged[1:]
merged = merged.append(play_3)
merged = merged.append(play_4)
merged = merged.append(play_5)
merged = merged.append(play_6)
merged = merged.append(play_7)

player_traditional_box_scores_2018_19 = merged
player_traditional_box_scores_2018_19.to_excel('data_outputs\\player_traditional_box_scores_2018_19.xlsx')

In [ ]:
# 2017-18
play_1 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2017-18_sort_gdate_dir_-1_Month_1_SeasonType_Regular_20Season.xlsx')
play_2 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2017-18_sort_gdate_dir_-1_Month_2_SeasonType_Regular_20Season.xlsx')
play_3 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2017-18_sort_gdate_dir_-1_Month_3_SeasonType_Regular_20Season.xlsx')
play_4 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2017-18_sort_gdate_dir_-1_Month_4_SeasonType_Regular_20Season.xlsx')
play_5 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2017-18_sort_gdate_dir_-1_Month_5_SeasonType_Regular_20Season.xlsx')
play_6 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2017-18_sort_gdate_dir_-1_Month_6_SeasonType_Regular_20Season.xlsx')
play_7 = pd.read_excel('player_box_scores\\boxscores-traditional__Season_2017-18_sort_gdate_dir_-1_Month_7_SeasonType_Regular_20Season.xlsx')

merged = play_1.append(play_2)
merged = merged[1:]
merged = merged.append(play_3)
merged = merged.append(play_4)
merged = merged.append(play_5)
merged = merged.append(play_6)
merged = merged.append(play_7)

player_traditional_box_scores_2017_18 = merged
player_traditional_box_scores_2017_18.to_excel('data_outputs\\player_traditional_box_scores_2017_18.xlsx')

In [ ]:
player_traditional_box_scores_2019_20.head()

In [ ]:
player_tb = [player_traditional_box_scores_2020_21, player_traditional_box_scores_2019_20, player_traditional_box_scores_2018_19,
            player_traditional_box_scores_2017_18]

In [ ]:
y = 20

for year in player_tb:
    year['team_date_id'] = year['Team'] + '_' + year['Game\xa0Date']
    for team in teams:
        player_boxes = year.loc[year['Team'] == team]
        player_boxes.to_excel('team_players\\'+ str(team) +'_player_boxes_' + str(y) +'.xlsx')
    y -= 1
    print (f' {year} done!')    

In [ ]:
y = 20

for year in player_tb:
    for team in teams:
        cols = []
        player_boxes = year.loc[year['Team'] == team]
        player_namez = player_boxes['Player'].unique()        # Get unique player names
        for player in player_namez:                       # Add players as columns
            player_boxes[str(player) + '_played'] = np.where( player_boxes['Player'] == str(player), 1, 0)
            cols1 = str(player) + '_played'
            cols.append(cols1)
        grouped = player_boxes.groupby('team_date_id')[cols].sum()
        grouped.to_excel('team_players\\'+ str(team) +'_whoplayed_'+ str(y) + '.xlsx')
    y -= 1
    print (f' {y} done!')    

## Team Playtypes

In [ ]:
#os.mkdir('team_playtype_outputs')

In [ ]:
# https://www.nba.com/stats/players/isolation/

years = ['2021-22', '2020-21', '2019-20', '2018-19', '2017-18']
playtypes = ['isolation', 'transition', 'ball-handler', 'roll-man', 
            'playtype-post-up','spot-up', 'hand-off', 'cut',
            'off-screen', 'putbacks', 'misc'] 
season_types = ['Playoffs', 'Regular%20Season']

In [ ]:
plays_21 = pd.read_excel('team_playtype\\isolation_SeasonType_Regular_20Season_Season_2021-22.xlsx')

In [ ]:
path = 'C:\\Users\\tmcro\\OneDrive\\0_NBA_Statistitcs_2021\\NBA Current Studies\\Flatiron Phase 3 Project\\team_playtype'
playtype_files = os.listdir(path)
playtype_files

In [ ]:
# get names
'''
for file in playtype_files:
    df = pd.read_excel('team_playtype\\' + file)
    chars = file.find('_')
    df['playtype'] = file[:chars]
    df.to_excel('team_playtype_outputs\\' + file)
'''

In [ ]:
path = 'C:\\Users\\tmcro\\OneDrive\\0_NBA_Statistitcs_2021\\NBA Current Studies\\Flatiron Phase 3 Project\\team_playtype_outputs'
playtype_files3 = os.listdir(path)
playtype_files3
pf3 = pd.DataFrame(playtype_files3)

In [ ]:
pf_reg = pf3.loc[pf3[0].astype(str).str.contains('2021-22' and 'Regular')]

### Append

In [ ]:
appended_data = []

for file in pf_reg[0]:
    data = pd.read_excel('team_playtype_outputs\\' + file)
    appended_data.append(data)

appended_data = pd.concat(appended_data)
appended_data

In [ ]:
appended_data.columns

In [ ]:
appended_data = appended_data[1:]

In [ ]:
appended_data = appended_data.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'])

In [ ]:
appended_data.head()

In [ ]:
#appended_data.to_excel('team_playtype_outputs\\appended_all_playtypes_regular_season.xlsx')

In [ ]:
appended_data = pd.read_excel('team_playtype_outputs\\appended_all_playtypes_regular_season.xlsx')
appended_data['Freq'] = appended_data['Freq'].str.rstrip("%").astype(float)/100
appended_data

In [ ]:
# Lets check out the data
ap_2021 = appended_data.loc[appended_data['season'] == '2021-22']
ap_2020 = appended_data.loc[appended_data['season'] == '2020-21']
ap_2019 = appended_data.loc[appended_data['season'] == '2019-20']
ap_2018 = appended_data.loc[appended_data['season'] == '2018-19']
ap_2017 = appended_data.loc[appended_data['season'] == '2017-18']

In [ ]:
ppp_21 = ap_2021.groupby('playtype')['PPP'].mean()
ppp_21 = pd.DataFrame(ppp_21)

In [ ]:

freq_21 = ap_2021.groupby('playtype')['Freq'].mean()
freq_21 = pd.DataFrame(freq_21)

In [ ]:
freq_21.round(2)

In [ ]:
ppp_21.reset_index(inplace = True)
ppp_21

In [ ]:
apz = [ap_2020, ap_2019, ap_2018, ap_2017]

i = 2020
for ap in apz:
    res = ap.groupby('playtype')['PPP'].mean()
    resy = pd.DataFrame(res)
    resy.reset_index(inplace = True)
    nm= str(i) + '_PPP'
    resy = resy.rename(columns = {'PPP' : nm})
    ppp_21 = pd.merge(ppp_21, resy, how = 'outer', on = 'playtype')
    i -= 1
ppp_21.round(2)


In [ ]:
ppp_21 = ppp_21.round(2)
ppp_21 = ppp_21.set_index('playtype')

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
ax = sns.heatmap(ppp_21, linewidths=.5)

ax.set_title('Points Per Play, 2017-2022', size = 20)

### Best plays?


In [ ]:
bp = appended_data.sort_values('PPP', ascending= False)
bp.head(10)

In [ ]:
bp.drop(columns = ['PTS', 'FGM', 'FGA', 'FG%', 'FTFreq', 'SFFreq', 'ScoreFreq'], inplace = True)
bp.dropna(inplace = True)

In [ ]:
bp.head(5)

## Combine all advanced box scores

In [ ]:
path = 'C:\\Users\\tmcro\\OneDrive\\0_NBA_Statistitcs_2021\\NBA Current Studies\\Flatiron Phase 3 Project\\team_boxes'
p = os.listdir(path)
pf = pd.DataFrame(p)

In [ ]:
# Get just the ones you want based on file names
pf_reg = pf.loc[pf[0].astype(str).str.contains('Regular')]
pf_reg = pf_reg.loc[pf_reg[0].astype(str).str.contains('advanced')]
pf_reg

In [ ]:
# append the data
advanced_boxes_17_21 = []

for file in pf_reg[0]:
    data = pd.read_excel('team_boxes\\' + file)
    advanced_boxes_17_21.append(data)

advanced_boxes_17_21 = pd.concat(advanced_boxes_17_21)
advanced_boxes_17_21

In [ ]:
advanced_boxes_17_21['wins'] = np.where(advanced_boxes_17_21['W/L'] == 'W', 1, 0)

In [ ]:
advanced_boxes_17_21.to_excel('team_boxes\\advanced_boxes_17_21.xlsx')

In [ ]:
adv_21 = pd.read_excel('team_general\\advanced_SeasonType_Regular_20Season_Season_2021-22.xlsx')
adv_20 = pd.read_excel('team_general\\advanced_SeasonType_Regular_20Season_Season_2020-21.xlsx')
adv_19 = pd.read_excel('team_general\\advanced_SeasonType_Regular_20Season_Season_2019-20.xlsx')
adv_18 = pd.read_excel('team_general\\advanced_SeasonType_Regular_20Season_Season_2018-19.xlsx')
adv_17 = pd.read_excel('team_general\\advanced_SeasonType_Regular_20Season_Season_2017-18.xlsx')

In [ ]:
advs = [adv_17, adv_18, adv_19, adv_20, adv_21]

for ad in advs:
    # Add team badge links
    ad['path'] = 'images/' + ad['TEAM'] + '.png'
    ad = ad[1:]


## Individual Player Defense

In [ ]:
# 3-pt Defense
d_3_21 = pd.read_excel('player_d_dash\defense-dash-3pt_SeasonType_Regular_20Season_Season_2021-22.xlsx')
d_3_21 =d_3_21[1:]
d_3_21.head()

In [ ]:
#Calculate Deciles
d_3_21['3D_Decile'] =  pd.qcut(d_3_21['DFG%'], 10, labels = False)
d_3_21.head()

In [ ]:
# Calculate Deciles by Team
team_3d_21 = d_3_21.groupby('Team')['3D_Decile'].agg(['mean', 'std', 'median'])
team_3d_21 = team_3d_21.round(2)
team_3d_21 = team_3d_21.sort_values('mean')
team_3d_21

In [ ]:
d_2_21 = pd.read_excel('player_d_dash\defense-dash-2pt_SeasonType_Regular_20Season_Season_2021-22.xlsx')
d_2_21 =d_2_21[1:]
d_2_21.head()

In [ ]:
d_2_21['2D_Decile'] =  pd.qcut(d_2_21['DFG%'], 10, labels = False)
d_2_21.head()

In [ ]:
# Calculate 2-pt Deciles by Team
team_2d_21 = d_2_21.groupby('Team')['2D_Decile'].agg(['mean', 'std', 'median'])
team_2d_21 = team_2d_21.round(2)
team_2d_21 = team_2d_21.sort_values('mean')
team_2d_21

In [ ]:
team_2d_21 = team_2d_21.rename(columns = {'mean' : '2p_mean', 'std' : '2p_std', 'median': '2p_median'})
team_3d_21 = team_3d_21.rename(columns = {'mean' : '3p_mean', 'std' : '3p_std', 'median': '3p_median'})

In [ ]:
team_3d_21.reset_index(inplace = True)
team_2d_21.reset_index(inplace = True)

In [ ]:
playerD_by_team_21 = pd.merge(team_2d_21, team_3d_21, on = 'Team', how = 'left')

In [ ]:
playerD_by_team_21

In [ ]:
playerD_by_team_21['team_avg_mean'] = (playerD_by_team_21['2p_mean'] + playerD_by_team_21['3p_mean']) /2


In [ ]:
playerD_by_team_21.sort_values('team_avg_mean', inplace = True)
playerD_by_team_21

### 2020 Player Defense

In [ ]:
# 3-pt Defense
d_3_20 = pd.read_excel('player_d_dash\defense-dash-3pt_SeasonType_Regular_20Season_Season_2020-21.xlsx')
d_3_20 =d_3_20[1:]
#Calculate Deciles
d_3_20['3D_Decile'] =  pd.qcut(d_3_20['DFG%'], 10, labels = False)
d_3_20.head()
# Calculate Deciles by Team
team_3d_20 = d_3_20.groupby('Team')['3D_Decile'].agg(['mean', 'std', 'median'])
team_3d_20 = team_3d_20.round(2)
team_3d_20 = team_3d_20.sort_values('mean')
# 2pt Defense
d_2_20 = pd.read_excel('player_d_dash\defense-dash-2pt_SeasonType_Regular_20Season_Season_2021-22.xlsx')
d_2_20 =d_2_20[1:]
d_2_20['2D_Decile'] =  pd.qcut(d_2_20['DFG%'], 10, labels = False)
# Calculate 2-pt Deciles by Team
team_2d_20 = d_2_20.groupby('Team')['2D_Decile'].agg(['mean', 'std', 'median'])
team_2d_20 = team_2d_20.round(2)
team_2d_20 = team_2d_20.sort_values('mean')
# rename columns
team_2d_20 = team_2d_20.rename(columns = {'mean' : '2p_mean', 'std' : '2p_std', 'median': '2p_median'})
team_3d_20 = team_3d_20.rename(columns = {'mean' : '3p_mean', 'std' : '3p_std', 'median': '3p_median'})
team_3d_20.reset_index(inplace = True)
team_2d_20.reset_index(inplace = True)
playerD_by_team_20 = pd.merge(team_2d_20, team_3d_20, on = 'Team', how = 'left')
playerD_by_team_20['team_avg_mean'] = (playerD_by_team_20['2p_mean'] + playerD_by_team_20['3p_mean']) /2
playerD_by_team_20.sort_values('team_avg_mean', inplace = True)
playerD_by_team_20



In [ ]:
#os.mkdir('player_d_outputs')

In [ ]:
# Save Individual Player Defense

d_2_21.to_excel('player_d_outputs\\2021_2ptD.xlsx')
d_3_21.to_excel('player_d_outputs\\2021_3ptD.xlsx')
d_2_20.to_excel('player_d_outputs\\2020_2ptD.xlsx')
d_3_20.to_excel('player_d_outputs\\2020_3ptD.xlsx')

In [ ]:
# 3-pt Defense
d_3_19 = pd.read_excel('player_d_dash\defense-dash-3pt_SeasonType_Regular_20Season_Season_2019-20.xlsx')
d_3_19 =d_3_19[1:]
#Calculate Deciles
d_3_19['3D_Decile'] =  pd.qcut(d_3_19['DFG%'], 10, labels = False)
d_3_19.head()
# Calculate Deciles by Team
team_3d_19 = d_3_19.groupby('Team')['3D_Decile'].agg(['mean', 'std', 'median'])
team_3d_19 = team_3d_19.round(2)
team_3d_19 = team_3d_19.sort_values('mean')
# 2pt Defense
d_2_19 = pd.read_excel('player_d_dash\defense-dash-2pt_SeasonType_Regular_20Season_Season_2021-22.xlsx')
d_2_19 =d_2_19[1:]
d_2_19['2D_Decile'] =  pd.qcut(d_2_19['DFG%'], 10, labels = False)
# Calculate 2-pt Deciles by Team
team_2d_19 = d_2_19.groupby('Team')['2D_Decile'].agg(['mean', 'std', 'median'])
team_2d_19 = team_2d_19.round(2)
team_2d_19 = team_2d_19.sort_values('mean')
# rename columns
team_2d_19 = team_2d_19.rename(columns = {'mean' : '2p_mean', 'std' : '2p_std', 'median': '2p_median'})
team_3d_19 = team_3d_19.rename(columns = {'mean' : '3p_mean', 'std' : '3p_std', 'median': '3p_median'})
team_3d_19.reset_index(inplace = True)
team_2d_19.reset_index(inplace = True)
playerD_by_team_19 = pd.merge(team_2d_19, team_3d_19, on = 'Team', how = 'left')
playerD_by_team_19['team_avg_mean'] = (playerD_by_team_19['2p_mean'] + playerD_by_team_19['3p_mean']) /2
playerD_by_team_19.sort_values('team_avg_mean', inplace = True)
playerD_by_team_19

In [ ]:
d_2_19.to_excel('player_d_outputs\\2019_2ptD.xlsx')
d_3_19.to_excel('player_d_outputs\\2019_3ptD.xlsx')

In [ ]:
# 3-pt Defense
d_3_18 = pd.read_excel('player_d_dash\defense-dash-3pt_SeasonType_Regular_20Season_Season_2018-19.xlsx')
d_3_18 =d_3_18[1:]
#Calculate Deciles
d_3_18['3D_Decile'] =  pd.qcut(d_3_18['DFG%'], 10, labels = False)
d_3_18.head()
# Calculate Deciles by Team
team_3d_18 = d_3_18.groupby('Team')['3D_Decile'].agg(['mean', 'std', 'median'])
team_3d_18 = team_3d_18.round(2)
team_3d_18 = team_3d_18.sort_values('mean')
# 2pt Defense
d_2_18 = pd.read_excel('player_d_dash\defense-dash-2pt_SeasonType_Regular_20Season_Season_2021-22.xlsx')
d_2_18 =d_2_18[1:]
d_2_18['2D_Decile'] =  pd.qcut(d_2_18['DFG%'], 10, labels = False)
# Calculate 2-pt Deciles by Team
team_2d_18 = d_2_18.groupby('Team')['2D_Decile'].agg(['mean', 'std', 'median'])
team_2d_18 = team_2d_18.round(2)
team_2d_18 = team_2d_18.sort_values('mean')
# rename columns
team_2d_18 = team_2d_18.rename(columns = {'mean' : '2p_mean', 'std' : '2p_std', 'median': '2p_median'})
team_3d_18 = team_3d_18.rename(columns = {'mean' : '3p_mean', 'std' : '3p_std', 'median': '3p_median'})
team_3d_18.reset_index(inplace = True)
team_2d_18.reset_index(inplace = True)
playerD_by_team_18 = pd.merge(team_2d_18, team_3d_18, on = 'Team', how = 'left')
playerD_by_team_18['team_avg_mean'] = (playerD_by_team_18['2p_mean'] + playerD_by_team_18['3p_mean']) /2
playerD_by_team_18.sort_values('team_avg_mean', inplace = True)
d_2_18.to_excel('player_d_outputs\\2018_2ptD.xlsx')
d_3_18.to_excel('player_d_outputs\\2018_3ptD.xlsx')
playerD_by_team_18

In [ ]:
# 3-pt Defense
d_3_17 = pd.read_excel('player_d_dash\defense-dash-3pt_SeasonType_Regular_20Season_Season_2017-18.xlsx')
d_3_17 =d_3_17[1:]
#Calculate Deciles
d_3_17['3D_Decile'] =  pd.qcut(d_3_17['DFG%'], 10, labels = False)
d_3_17.head()
# Calculate Deciles by Team
team_3d_17 = d_3_17.groupby('Team')['3D_Decile'].agg(['mean', 'std', 'median'])
team_3d_17 = team_3d_17.round(2)
team_3d_17 = team_3d_17.sort_values('mean')
# 2pt Defense
d_2_17 = pd.read_excel('player_d_dash\defense-dash-2pt_SeasonType_Regular_20Season_Season_2021-22.xlsx')
d_2_17 =d_2_17[1:]
d_2_17['2D_Decile'] =  pd.qcut(d_2_17['DFG%'], 10, labels = False)
# Calculate 2-pt Deciles by Team
team_2d_17 = d_2_17.groupby('Team')['2D_Decile'].agg(['mean', 'std', 'median'])
team_2d_17 = team_2d_17.round(2)
team_2d_17 = team_2d_17.sort_values('mean')
# rename columns
team_2d_17 = team_2d_17.rename(columns = {'mean' : '2p_mean', 'std' : '2p_std', 'median': '2p_median'})
team_3d_17 = team_3d_17.rename(columns = {'mean' : '3p_mean', 'std' : '3p_std', 'median': '3p_median'})
team_3d_17.reset_index(inplace = True)
team_2d_17.reset_index(inplace = True)
playerD_by_team_17 = pd.merge(team_2d_17, team_3d_17, on = 'Team', how = 'left')
playerD_by_team_17['team_avg_mean'] = (playerD_by_team_17['2p_mean'] + playerD_by_team_17['3p_mean']) /2
playerD_by_team_17.sort_values('team_avg_mean', inplace = True)
d_2_17.to_excel('player_d_outputs\\2017_2ptD.xlsx')
d_3_17.to_excel('player_d_outputs\\2017_3ptD.xlsx')
playerD_by_team_17
playerD_by_team_17

# Player Data

## Shooting Deciles

### Download Traditional General Player Data

In [2]:
shooting_21 = pd.read_excel('scraped_data\\traditional_SeasonType_Regular_20Season_Season_2021-22.xlsx', thousands=',')
shooting_20 = pd.read_excel('scraped_data\\traditional_SeasonType_Regular_20Season_Season_2020-21.xlsx', thousands=',')
shooting_19 = pd.read_excel('scraped_data\\traditional_SeasonType_Regular_20Season_Season_2019-20.xlsx', thousands=',')
shooting_18 = pd.read_excel('scraped_data\\traditional_SeasonType_Regular_20Season_Season_2018-19.xlsx', thousands=',')
shooting_17 = pd.read_excel('scraped_data\\traditional_SeasonType_Regular_20Season_Season_2017-18.xlsx', thousands=',')

In [3]:
shooting_yrs = [shooting_21, shooting_20, shooting_19, shooting_18, shooting_17]

In [4]:
shooting_21 = shooting_21[1:]

In [5]:
shooting_21.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'PLAYER', 'TEAM', 'AGE', 'GP', 'W', 'L',
       'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA',
       'FT%', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'PF', 'FP',
       'DD2', 'TD3', '+/-'],
      dtype='object')

In [6]:
''' 
shooting_21['3p%_decile'] = pd.qcut(shooting_21['3P%'], 10, labels = False, duplicates = 'drop')
shooting_21['FG%_decile'] = pd.qcut(shooting_21['FG%'], 10, labels = False)
shooting_21['PPG_decile'] = pd.qcut(shooting_21['PTS'], 10, labels = False)

shooting_21
'''

" \nshooting_21['3p%_decile'] = pd.qcut(shooting_21['3P%'], 10, labels = False, duplicates = 'drop')\nshooting_21['FG%_decile'] = pd.qcut(shooting_21['FG%'], 10, labels = False)\nshooting_21['PPG_decile'] = pd.qcut(shooting_21['PTS'], 10, labels = False)\n\nshooting_21\n"

In [7]:
shooting_21

,Unnamed: 0,Unnamed: 1,PLAYER,TEAM,AGE,GP,W,L,MIN,PTS,...,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-
1,1,1.0,Joel Embiid,PHI,28.0,68.0,45.0,23.0,33.8,30.6,...,11.7,4.2,3.1,1.1,1.5,2.7,55.5,46.0,2.0,5.4
2,2,2.0,LeBron James,LAL,37.0,56.0,25.0,31.0,37.2,30.3,...,8.2,6.2,3.5,1.3,1.1,2.2,53.0,21.0,6.0,-2.1
3,3,3.0,Giannis Antetokounmpo,MIL,27.0,67.0,45.0,22.0,32.9,29.9,...,11.6,5.8,3.3,1.1,1.4,3.2,56.5,46.0,4.0,5.9
4,4,4.0,Kevin Durant,BKN,33.0,55.0,36.0,19.0,37.2,29.9,...,7.4,6.4,3.5,0.9,0.9,2.1,50.3,18.0,4.0,4.9
5,5,5.0,Luka Doncic,DAL,23.0,65.0,44.0,21.0,35.4,28.4,...,9.1,8.7,4.5,1.2,0.6,2.2,53.1,44.0,10.0,2.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,601,582.0,Nate Hinton,IND,23.0,2.0,0.0,2.0,1.2,0.0,...,0.0,0.0,0.5,0.0,0.0,0.0,-0.5,0.0,0.0,2.0
602,602,582.0,Sam Dekker,TOR,28.0,1.0,1.0,0.0,0.9,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0
603,603,582.0,Trayvon Palmer,DET,27.0,1.0,0.0,1.0,16.8,0.0,...,2.0,0.0,1.0,0.0,0.0,2.0,1.4,0.0,0.0,-12.0
604,604,582.0,Tyler Hall,NYK,25.0,1.0,1.0,0.0,1.9,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0


## True Shooting Deciles

In [8]:
trueShoot21 = pd.read_excel('scraped_data\\advanced_SeasonType_Regular_20Season_Season_2021-22.xlsx', thousands=',')
trueShoot20 = pd.read_excel('scraped_data\\advanced_SeasonType_Regular_20Season_Season_2020-21.xlsx', thousands=',')
trueShoot19 = pd.read_excel('scraped_data\\advanced_SeasonType_Regular_20Season_Season_2019-20.xlsx', thousands=',')
trueShoot18 = pd.read_excel('scraped_data\\advanced_SeasonType_Regular_20Season_Season_2018-19.xlsx', thousands=',')
trueShoot17 = pd.read_excel('scraped_data\\advanced_SeasonType_Regular_20Season_Season_2017-18.xlsx', thousands=',')

In [9]:
trueShootings = [trueShoot21, trueShoot20, trueShoot19, trueShoot18, trueShoot17]

In [10]:
trueShoot21 = trueShoot21[1:]

In [11]:
trueShoot21.dtypes

Unnamed: 0      int64
Unnamed: 1    float64
PLAYER         object
TEAM           object
AGE           float64
GP            float64
W             float64
L             float64
MIN           float64
OFFRTG        float64
DEFRTG        float64
NETRTG        float64
AST%          float64
AST/TO        float64
AST Ratio     float64
OREB%         float64
DREB%         float64
REB%          float64
TO Ratio      float64
eFG%          float64
TS%           float64
USG%          float64
PACE          float64
PIE           float64
POSS          float64
dtype: object

#### Add shooting to true shooting before decile calcs

In [12]:
trueShoot21 = pd.merge(trueShoot21, shooting_21, on = 'PLAYER', how = 'left')
trueShoot20 = pd.merge(trueShoot20, shooting_20, on = 'PLAYER', how = 'left')
trueShoot19 = pd.merge(trueShoot19, shooting_19, on = 'PLAYER', how = 'left')
trueShoot18 = pd.merge(trueShoot18, shooting_18, on = 'PLAYER', how = 'left')
trueShoot17 = pd.merge(trueShoot17, shooting_17, on = 'PLAYER', how = 'left')

### Add defense

In [13]:
trey_D_21 = pd.read_excel('player_d_dash\\defense-dash-3pt_SeasonType_Regular_20Season_Season_2021-22.xlsx')
trey_D_20 = pd.read_excel('player_d_dash\\defense-dash-3pt_SeasonType_Regular_20Season_Season_2020-21.xlsx')
trey_D_19 = pd.read_excel('player_d_dash\\defense-dash-3pt_SeasonType_Regular_20Season_Season_2019-20.xlsx')
trey_D_18 = pd.read_excel('player_d_dash\\defense-dash-3pt_SeasonType_Regular_20Season_Season_2020-21.xlsx')
trey_D_17 = pd.read_excel('player_d_dash\\defense-dash-3pt_SeasonType_Regular_20Season_Season_2018-19.xlsx')

In [14]:
keep = ['Player', 'FREQ', 'DFGM', 'DFGA', 'DFG%']
trey_D_17 = trey_D_17[keep].rename(columns = {'FREQ' : '3ptD_Freq', 'DFGM' : '3ptD_FGM',
                                                'DFGA' : '3ptD_FGA', 'DFG%' : '3PtD_FG%' })



In [15]:
trey_D_18 = trey_D_18[keep].rename(columns = {'FREQ' : '3ptD_Freq', 'DFGM' : '3ptD_FGM',
                                                'DFGA' : '3ptD_FGA', 'DFG%' : '3PtD_FG%' })

trey_D_19 = trey_D_19[keep].rename(columns = {'FREQ' : '3ptD_Freq', 'DFGM' : '3ptD_FGM',
                                                'DFGA' : '3ptD_FGA', 'DFG%' : '3PtD_FG%' })

trey_D_20 = trey_D_20[keep].rename(columns = {'FREQ' : '3ptD_Freq', 'DFGM' : '3ptD_FGM',
                                                'DFGA' : '3ptD_FGA', 'DFG%' : '3PtD_FG%' })

trey_D_21 = trey_D_21[keep].rename(columns = {'FREQ' : '3ptD_Freq', 'DFGM' : '3ptD_FGM',
                                                'DFGA' : '3ptD_FGA', 'DFG%' : '3PtD_FG%' })

In [16]:
alldata21 = pd.merge(trueShoot21, trey_D_21, left_on='PLAYER', right_on= 'Player', how= 'left')
alldata20 = pd.merge(trueShoot20, trey_D_20, left_on='PLAYER', right_on= 'Player', how= 'left')
alldata19 = pd.merge(trueShoot19, trey_D_19, left_on='PLAYER', right_on= 'Player', how= 'left')
alldata18 = pd.merge(trueShoot18, trey_D_18, left_on='PLAYER', right_on= 'Player', how= 'left')
alldata17 = pd.merge(trueShoot17, trey_D_17, left_on='PLAYER', right_on= 'Player', how= 'left')

In [17]:
alldata17

,Unnamed: 0_x,Unnamed: 1_x,PLAYER,TEAM_x,AGE_x,GP_x,W_x,L_x,MIN_x,OFFRTG,...,PF,FP,DD2,TD3,+/-,Player,3ptD_Freq,3ptD_FGM,3ptD_FGA,3PtD_FG%
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1.0,Andrew Wiggins,MIN,23.0,82.0,47.0,35.0,36.3,112.5,...,2.0,29.4,1.0,0.0,2.5,Andrew Wiggins,38.4%,1.9,5.2,36.1
2,2,1.0,Bismack Biyombo,ORL,25.0,82.0,25.0,57.0,18.2,97.9,...,1.9,17.0,7.0,0.0,-5.2,Bismack Biyombo,25.1%,0.7,2.0,35.6
3,3,1.0,Bradley Beal,WAS,25.0,82.0,43.0,39.0,36.3,108.7,...,2.0,37.0,4.0,0.0,2.2,Bradley Beal,40.3%,2.1,5.4,38.1
4,4,1.0,Cory Joseph,IND,26.0,82.0,48.0,34.0,27.0,105.4,...,1.7,19.0,1.0,0.0,-0.6,Cory Joseph,35.5%,1.2,3.5,34.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,536,524.0,Mindaugas Kuzminskas,NYK,28.0,1.0,0.0,1.0,1.9,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
537,537,524.0,Naz Mitrou-Long,UTA,24.0,1.0,0.0,1.0,0.5,250.0,...,0.0,3.0,0.0,0.0,5.0,Naz Mitrou-Long,38.9%,0.5,1.2,42.9
538,538,524.0,Scotty Hopson,DAL,28.0,1.0,0.0,1.0,8.4,77.3,...,0.0,1.5,0.0,0.0,-8.0,NaN,NaN,NaN,NaN,NaN
539,539,524.0,Trey McKinney-Jones,IND,27.0,1.0,1.0,0.0,1.2,100.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


### Add within 10-feet defense

In [18]:
in_D_21 = pd.read_excel('player_d_dash\\defense-dash-lt10_SeasonType_Regular_20Season_Season_2021-22.xlsx')
in_D_20 = pd.read_excel('player_d_dash\\defense-dash-lt10_SeasonType_Regular_20Season_Season_2020-21.xlsx')
in_D_19 = pd.read_excel('player_d_dash\\defense-dash-lt10_SeasonType_Regular_20Season_Season_2019-20.xlsx')
in_D_18 = pd.read_excel('player_d_dash\\defense-dash-lt10_SeasonType_Regular_20Season_Season_2020-21.xlsx')
in_D_17 = pd.read_excel('player_d_dash\\defense-dash-lt10_SeasonType_Regular_20Season_Season_2018-19.xlsx')

In [19]:
keep = ['Player', 'FREQ', 'DFGM', 'DFGA', 'DFG%']
in_D_17 = in_D_17[keep].rename(columns = {'FREQ' : 'ins_10_D_Freq', 'DFGM' : 'ins_10_D_FGM',
                                                'DFGA' : 'ins_10_D_FGA', 'DFG%' : 'ins_10_D_FG%' })

in_D_17

,Player,ins_10_D_Freq,ins_10_D_FGM,ins_10_D_FGA,ins_10_D_FG%
0,NaN,NaN,NaN,NaN,NaN
1,Rudy Gobert,51.0%,5.4,10.7,50.7
2,Joel Embiid,47.4%,5.0,10.6,47.4
3,Myles Turner,61.1%,5.2,10.2,51.3
4,Nikola Jokic,54.9%,5.3,9.8,54.3
...,...,...,...,...,...
515,Brandon Goodwin,44.4%,0.6,0.7,75.0
516,Jaylen Morris,25.0%,0.0,0.7,0.0
517,Donatas Motiejunas,33.3%,0.3,0.7,50.0
518,Kostas Antetokounmpo,16.7%,0.0,0.5,0.0


In [20]:
in_D_18 = in_D_18[keep].rename(columns = {'FREQ' : 'ins_10_D_Freq', 'DFGM' : 'ins_10_D_FGM',
                                                'DFGA' : 'ins_10_D_FGA', 'DFG%' : 'ins_10_D_FG%' })

in_D_19 = in_D_19[keep].rename(columns = {'FREQ' : 'ins_10_D_Freq', 'DFGM' : 'ins_10_D_FGM',
                                                'DFGA' : 'ins_10_D_FGA', 'DFG%' : 'ins_10_D_FG%' })

in_D_20 = in_D_20[keep].rename(columns = {'FREQ' : 'ins_10_D_Freq', 'DFGM' : 'ins_10_D_FGM',
                                                'DFGA' : 'ins_10_D_FGA', 'DFG%' : 'ins_10_D_FG%' })

in_D_21 = in_D_21[keep].rename(columns = {'FREQ' : 'ins_10_D_Freq', 'DFGM' : 'ins_10_D_FGM',
                                                'DFGA' : 'ins_10_D_FGA', 'DFG%' : 'ins_10_D_FG%' })

In [21]:
alldata21 = pd.merge(alldata21, in_D_21, left_on='PLAYER', right_on= 'Player', how= 'left')
alldata20 = pd.merge(alldata20, in_D_20, left_on='PLAYER', right_on= 'Player', how= 'left')
alldata19 = pd.merge(alldata19, in_D_19, left_on='PLAYER', right_on= 'Player', how= 'left')
alldata18 = pd.merge(alldata18, in_D_18, left_on='PLAYER', right_on= 'Player', how= 'left')
alldata17 = pd.merge(alldata17, in_D_17, left_on='PLAYER', right_on= 'Player', how= 'left')


In [22]:
alldata19

,Unnamed: 0_x,Unnamed: 1_x,PLAYER,TEAM_x,AGE_x,GP_x,W_x,L_x,MIN_x,OFFRTG,...,Player_x,3ptD_Freq,3ptD_FGM,3ptD_FGA,3PtD_FG%,Player_y,ins_10_D_Freq,ins_10_D_FGM,ins_10_D_FGA,ins_10_D_FG%
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1.0,Maxi Kleber,DAL,28.0,74.0,42.0,32.0,25.5,112.6,...,Maxi Kleber,27.9%,1.2,3.6,32.0,Maxi Kleber,52.1%,3.6,6.8,52.5
2,2,2.0,Delon Wright,DAL,28.0,73.0,42.0,31.0,21.5,111.2,...,Delon Wright,31.2%,0.8,2.4,33.9,Delon Wright,53.0%,2.4,4.1,58.1
3,3,2.0,Dillon Brooks,MEM,24.0,73.0,34.0,39.0,28.9,108.5,...,Dillon Brooks,44.0%,2.0,5.2,37.9,Dillon Brooks,39.8%,2.5,4.7,54.0
4,4,2.0,Duncan Robinson,MIA,26.0,73.0,44.0,29.0,29.7,113.8,...,Duncan Robinson,43.8%,1.5,4.5,33.5,Duncan Robinson,44.7%,2.7,4.6,59.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,525,524.0,Eric Mika,SAC,25.0,1.0,0.0,1.0,19.0,122.0,...,Eric Mika,37.5%,2.0,3.0,66.7,Eric Mika,37.5%,2.0,3.0,66.7
526,526,524.0,J.P. Macura,CLE,25.0,1.0,1.0,0.0,0.6,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527,527,524.0,Jamal Crawford,BKN,40.0,1.0,1.0,0.0,6.0,133.3,...,Jamal Crawford,100.0%,1.0,1.0,100.0,NaN,NaN,NaN,NaN,NaN
528,528,524.0,Malik Newman,CLE,23.0,1.0,0.0,1.0,3.7,57.1,...,NaN,NaN,NaN,NaN,NaN,Malik Newman,50.0%,1.0,1.0,100.0


#### Filter out 
those who played less than 1000 possessions

In [23]:
alldata21 = alldata21.loc[alldata21['POSS'] > 1000]
alldata20 = alldata20.loc[alldata20['POSS'] > 1000]
alldata19 = alldata19.loc[alldata19['POSS'] > 1000]
alldata18 = alldata18.loc[alldata18['POSS'] > 1000]
alldata17 = alldata17.loc[alldata17['POSS'] > 1000]

### ADD the DECILES!

In [24]:
alldata21['TS_decile']= pd.qcut(alldata21['TS%'], 10, labels = False)
alldata21

,Unnamed: 0_x,Unnamed: 1_x,PLAYER,TEAM_x,AGE_x,GP_x,W_x,L_x,MIN_x,OFFRTG,...,3ptD_Freq,3ptD_FGM,3ptD_FGA,3PtD_FG%,Player_y,ins_10_D_Freq,ins_10_D_FGM,ins_10_D_FGA,ins_10_D_FG%,TS_decile
0,1,1.0,Deni Avdija,WAS,21.0,82.0,35.0,47.0,24.2,109.2,...,32.9%,1.3,4.1,32.3,Deni Avdija,50.4%,3.2,6.2,52.2,2
1,2,1.0,Dwight Powell,DAL,30.0,82.0,52.0,30.0,21.9,113.2,...,23.0%,0.9,2.7,33.9,Dwight Powell,54.1%,3.7,6.3,59.8,9
2,3,1.0,Kevon Looney,GSW,26.0,82.0,53.0,29.0,21.1,113.2,...,27.4%,1.1,3.3,33.7,Kevon Looney,46.6%,3.2,5.7,56.1,6
3,4,1.0,Mikal Bridges,PHX,25.0,82.0,64.0,18.0,34.8,116.7,...,37.0%,1.6,4.7,34.0,Mikal Bridges,44.3%,3.3,5.6,58.3,9
4,5,1.0,Saddiq Bey,DET,23.0,82.0,23.0,59.0,33.0,104.7,...,34.9%,1.1,2.9,36.8,Saddiq Bey,48.2%,2.3,4.0,56.7,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,415,414.0,T.J. McConnell,IND,30.0,27.0,9.0,18.0,24.2,106.8,...,34.4%,1.0,2.8,34.2,T.J. McConnell,46.6%,2.3,3.8,59.2,1
416,417,417.0,Derrick Rose,NYK,33.0,26.0,11.0,15.0,24.5,109.1,...,39.1%,1.1,3.7,28.9,Derrick Rose,41.5%,1.8,4.0,45.6,2
421,422,420.0,Nerlens Noel,NYK,28.0,25.0,10.0,15.0,22.5,104.1,...,27.4%,1.3,3.0,44.0,Nerlens Noel,55.5%,3.2,6.1,52.0,6
422,423,423.0,Brandon Williams,POR,22.0,24.0,2.0,22.0,26.7,101.2,...,44.2%,2.1,4.9,42.4,Brandon Williams,43.1%,2.7,4.8,56.5,0


### Save the alldata player files

In [26]:
# Save in final_player_data
alldata21.to_excel('final_player_data\\alldata21.xlsx')
alldata20.to_excel('final_player_data\\alldata20.xlsx')
alldata19.to_excel('final_player_data\\alldata19.xlsx')
alldata18.to_excel('final_player_data\\alldata18.xlsx')
alldata17.to_excel('final_player_data\\alldata17.xlsx')

## Add Deciles

USG, D_3pt%, D_within_10, 3pt%, Reb%, Ast%, TO%

In [30]:
list(alldata21.columns)

['Unnamed: 0_x',
 'Unnamed: 1_x',
 'PLAYER',
 'TEAM_x',
 'AGE_x',
 'GP_x',
 'W_x',
 'L_x',
 'MIN_x',
 'OFFRTG',
 'DEFRTG',
 'NETRTG',
 'AST%',
 'AST/TO',
 'AST\xa0Ratio',
 'OREB%',
 'DREB%',
 'REB%',
 'TO\xa0Ratio',
 'eFG%',
 'TS%',
 'USG%',
 'PACE',
 'PIE',
 'POSS',
 'Unnamed: 0_y',
 'Unnamed: 1_y',
 'TEAM_y',
 'AGE_y',
 'GP_y',
 'W_y',
 'L_y',
 'MIN_y',
 'PTS',
 'FGM',
 'FGA',
 'FG%',
 '3PM',
 '3PA',
 '3P%',
 'FTM',
 'FTA',
 'FT%',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'TOV',
 'STL',
 'BLK',
 'PF',
 'FP',
 'DD2',
 'TD3',
 '+/-',
 'Player_x',
 '3ptD_Freq',
 '3ptD_FGM',
 '3ptD_FGA',
 '3PtD_FG%',
 'Player_y',
 'ins_10_D_Freq',
 'ins_10_D_FGM',
 'ins_10_D_FGA',
 'ins_10_D_FG%',
 'TS_decile']

In [28]:
ad21 = pd.read_excel('final_player_data\\alldata21.xlsx')
ad20 = pd.read_excel('final_player_data\\alldata20.xlsx')
ad19 = pd.read_excel('final_player_data\\alldata19.xlsx')
ad18 = pd.read_excel('final_player_data\\alldata18.xlsx')
ad17 = pd.read_excel('final_player_data\\alldata17.xlsx')

tsts = [ad21, ad20, ad19, ad18, ad17]

In [31]:
i = 21
for t in tsts:
    t['TS_decile'] = pd.qcut(t['TS%'], 10, labels = False) 
    t['usg_decile'] = pd.qcut(t['USG%'], 10, labels = False) 
    t['3pt_Def_decile'] = pd.qcut(t['3PtD_FG%'], 10, labels = False) 
    t['Def_within10_decile'] = pd.qcut(t['ins_10_D_FG%'], 10, labels = False) 
    t['3pt_decile'] = pd.qcut(t['3P%'], 10, labels = False) 
    t['Reb_%_decile'] = pd.qcut(t['REB%'], 10, labels = False) 
    t['AST_%_decile'] = pd.qcut(t['AST%'], 10, labels = False) 
    t['TOV_%_decile'] = pd.qcut(t['TO\xa0Ratio'], 10, labels = False)
    i -= 1
    t.to_excel('final_player_data\\deciles_alldata'+ str(i) +'.xlsx')
    

## Yearly Player Data Together  (is this old?)

Here, I want to merge player_general_outputs/trueshoot and player_general_outputs/ppm

In [ ]:
# True shoot 
ts21 = pd.read_excel('player_general_outputs\\trueShoot21.xlsx')
ts20 = pd.read_excel('player_general_outputs\\trueShoot20.xlsx')
ts19 = pd.read_excel('player_general_outputs\\trueShoot19.xlsx')
ts18 = pd.read_excel('player_general_outputs\\trueShoot18.xlsx')
ts17 = pd.read_excel('player_general_outputs\\trueShoot17.xlsx')

#ppm
p21 = pd.read_excel('player_general_outputs\\ppm_reb_deciles_21.xlsx')
p20 = pd.read_excel('player_general_outputs\\ppm_reb_deciles_20.xlsx')
p19 = pd.read_excel('player_general_outputs\\ppm_reb_deciles_19.xlsx')
p18 = pd.read_excel('player_general_outputs\\ppm_reb_deciles_18.xlsx')
p17 = pd.read_excel('player_general_outputs\\ppm_reb_deciles_17.xlsx')

In [ ]:
ts21.shape

In [ ]:
p21.shape

In [ ]:
player_data_21 = pd.merge(ts21, p21, how = 'left', on= 'PLAYER')
player_data_20 = pd.merge(ts20, p20, how = 'left', on= 'PLAYER')
player_data_19 = pd.merge(ts19, p19, how = 'left', on= 'PLAYER')
player_data_18 = pd.merge(ts18, p18, how = 'left', on= 'PLAYER')
player_data_17 = pd.merge(ts17, p17, how = 'left', on= 'PLAYER')
player_data_17

In [ ]:
player_data_21.columns

In [ ]:
mehcols = ['Unnamed: 0_x', 'Unnamed: 0.1_x', 'Unnamed: 1_x']

In [ ]:
playeryears = [player_data_21, player_data_20, player_data_19, player_data_18, player_data_17]

In [ ]:
for p in playeryears:
    p.drop(columns = mehcols, inplace = True)


In [ ]:
player_data_21.to_excel('player_general_outputs\\player_data_2021.xlsx')
player_data_20.to_excel('player_general_outputs\\player_data_2020.xlsx')
player_data_19.to_excel('player_general_outputs\\player_data_2019.xlsx')
player_data_18.to_excel('player_general_outputs\\player_data_2018.xlsx')
player_data_17.to_excel('player_general_outputs\\player_data_2017.xlsx')


In [ ]:
# make folder for data outputs after python work
#os.mkdir('player_data_year_outputs')

In [ ]:
plr_data21.to_excel('2021_player_data_all.xlsx')
plr_data20.to_excel('2020_player_data_all.xlsx')
plr_data19.to_excel('2019_player_data_all.xlsx')
plr_data18.to_excel('2018_player_data_all.xlsx')
plr_data17.to_excel('2017_player_data_all.xlsx')

## Make a player algogrithm!

We have 
- True shooting decile, 
- Points per minute decile, 
- rebounding decile, 
- 3pt decile, 
- ast decile
- 3pt D decile
- inside 10ft decile

How to weigh them?

True shooting and three point shooting are both very important, but true shooting more so. rebounding decile is important, as are assists... 

Half of the game is defense, so 20% to 3pt decile and 20% to inside 10ft decile, and 10% to rebounding.

True shooting decile gets 25%, ast gets 15%, ppm is a flawed metric now that I think about it, I would rather it be posession-based. So, 3pt percentage decile gets 10%. 

In [ ]:
pd21 = plr_data21[plr_data21.columns[plr_data21.columns.str.contains("decile")]]
pd21

In [ ]:
pd21.columns

In [ ]:
colz = ['TS_decile', 'ppm_decile', 'reb_decile', '3p_decile', 'ast_decile',
       '3ptD_decile_x', '3ptD_decile_y', 'within_10ft_defense_decile',
       'PLAYER']
pd21= plr_data21[colz]

pd21

In [ ]:
plr_data21['trav_score'] = ((plr_data21['TS_decile'] * .25) + 
                        (plr_data21['3p_decile'] * .10) + 
                        (plr_data21['ast_decile'] * .15) +
                        ((10 - plr_data21['3ptD_decile_x']) * .20) +
                        ((10 - plr_data21['within_10ft_defense_decile']) * .20) +
                        ((10 - plr_data21['reb_decile']) * .10))




                        

In [ ]:
plrds = [plr_data21, plr_data20, plr_data19, plr_data18, plr_data17]


In [ ]:
for p in plrds:
    p['trav_score'] = ((p['TS_decile'] * .25) + 
                        (p['3p_decile'] * .10) + 
                        (p['ast_decile'] * .15) +
                        ((10 - p['3ptD_decile_x']) * .20) +
                        ((10 - p['within_10ft_defense_decile']) * .20) +
                        ((10 - p['reb_decile']) * .10))

In [ ]:
p_19= plr_data20.dropna()
p_19 = p_19.sort_values('trav_score', ascending = False)
p_19 = p_19.loc[p_19['MIN_x'] > 15]
p_19 = p_19.loc[:,~p_19.columns.duplicated()]
p_19 = p_19.loc[p_19['GP_x'] > 30]
p_19

In [ ]:
p_20 = plr_data20.dropna()
p_20 = p_20.sort_values('trav_score', ascending = False)
p_20 = p_20.loc[p_20['MIN_x'] > 15]
p_20 = p_20.loc[:,~p_20.columns.duplicated()]
p_20 = p_20.loc[p_20['GP_x'] > 30]
p_20


In [ ]:
por_20 = p_20.loc[p_20['TEAM_x'] == 'POR']
por_20

### Get rid of players with few games or minutes

In [ ]:
list(plr_data21.columns)

In [ ]:
plr_data21 = plr_data21.loc[:,~plr_data21.columns.duplicated()]
plr_data21

In [ ]:
plr_data21_filtered = plr_data21.loc[plr_data21['MIN_x'] >= 12]
plr_data21_filtered = plr_data21_filtered.loc[plr_data21_filtered['GP_x'] >= 30]
plr_data21_filtered

In [ ]:
plr_data21_por = plr_data21_filtered.loc[plr_data21_filtered['TEAM_x']== 'POR']
plr_data21_por

In [ ]:
pd21.sort_values('trav_score', ascending = False, inplace = True)
pd21

## Joining the big'ol dataframe

In [ ]:
list(team_box_21_comb_SA.columns)

In [ ]:
# The df as of yet is:
team_box_21_comb_SA['Home/Away'] = np.where(team_box_21_comb_SA['Match\xa0Up_x'].astype(str).str.contains('@'), 'Away', 'Home')

In [ ]:
team_box_21_comb_SA = team_box_21_comb_SA.drop(columns = ['indexcol','MIN_x','OffRtg',
                                    'DefRtg','NetRtg','AST%','AST/TO','ASTRatio',
                                    'OREB%_x','DREB%','REB%','TOV%_x',
                                    'eFG%_x','TS%','PACE','PIE',
                                    'Unnamed: 0_y', 'W/L_y', 'PTS', 'FGA'] )

team_box_21_comb_SA

In [ ]:
team_box_21_comb_SA = team_box_21_comb_SA.drop(columns = ['MIN_y','FG%','3PM','3PA','3P%',
                                    'FTM','FTA','FT%','OREB','DREB','REB','AST','TOV',
                                    'STL','BLK','PF','+/-','Unnamed: 0_x', 'Season_x.1',
                                     'MIN_x.1','%FGA2PT','%FGA3PT','%PTS2PT','%PTS2PT\xa0MR',
                                     '%PTS3PT','%PTSFBPs','%PTSFT','%PTSOff\xa0TO','%PTSPITP',
                                     '2FGM%AST','2FGM%UAST','3FGM%AST','3FGM%UAST','FGM%AST',
                                     'FGM%UAST','Game_date','Unnamed: 0_y.1','Team_y.1',
                                     'Match\xa0Up_y.1','Game\xa0Date_y.1','Season_y.1',
                                     'W/L_y.1','MIN_y.1','eFG%_y','FTARate','TOV%_y','OREB%_y',
                                     'OppeFG%','OppFTA\xa0Rate','OppTOV%','OppOREB%','Unnamed: 0',
                                     'Team','Match\xa0Up','Game\xa0Date','Season','W/L','MIN',
                                     'PTSOFF\xa0TO','2ndPTS','FBPs','PITP','Opp\xa0PTSOFF\xa0TO',
                                     'Opp2nd\xa0PTS','OppFBPs','OppPITP',] )
team_box_21_comb_SA

In [ ]:
final_box_21 = team_box_21_comb_SA.copy()

In [ ]:
#final_box_21.to_excel('data_outputs\\final_box_2021-2022.xlsx')

In [ ]:
final_box_21 = final_box_21.drop(columns = ['Season_y', 'FGM','Game\xa0Date_y', 'W/L_x.1',
                             'Season_y','FGM','Team_x.1','Match\xa0Up_x.1',
                              'Game\xa0Date_x.1','W/L_x.1'])

## Add Players Played in Game using Xlookup-like function to Final_Box

## Add players Defensive Ability (3pt shots)

In [ ]:
#New folder
#os.mkdir('aggregate_player_data') # for aggregates of individual stats


In [ ]:
#load playerdefense: 2021 defense
pd_21 = pd.read_excel('player_d_dash\defense-dash-3pt_SeasonType_Regular_20Season_Season_2021-22.xlsx')
pd_21 = pd_21[1:]

In [ ]:
pd_21['3ptD_decile'] = pd.qcut(pd_21['DFG%'], 10, labels = False)


In [ ]:
pd_21

In [ ]:
# 2017-2020
pd_17 = pd.read_excel('player_d_dash\defense-dash-3pt_SeasonType_Regular_20Season_Season_2017-18.xlsx')
pd_18 = pd.read_excel('player_d_dash\defense-dash-3pt_SeasonType_Regular_20Season_Season_2018-19.xlsx')
pd_19 = pd.read_excel('player_d_dash\defense-dash-3pt_SeasonType_Regular_20Season_Season_2019-20.xlsx')
pd_20 = pd.read_excel('player_d_dash\defense-dash-3pt_SeasonType_Regular_20Season_Season_2020-21.xlsx')

In [ ]:
pdz = [pd_17, pd_18, pd_19, pd_20, pd_21]


In [ ]:
n = 17
for p in pdz:
    p['3ptD_decile'] = pd.qcut(p['DFG%'], 10, labels = False)
    n += 1
    p.to_excel('aggregate_player_data\\3pt_defense_'+ str(n)  + '.xlsx')
    

## Players Defensive ability - within 10 feet of basket

In [ ]:
# 2017-2020
pd1_17 = pd.read_excel('player_d_dash\defense-dash-lt10_SeasonType_Regular_20Season_Season_2017-18.xlsx')
pd1_18 = pd.read_excel('player_d_dash\defense-dash-lt10_SeasonType_Regular_20Season_Season_2018-19.xlsx')
pd1_19 = pd.read_excel('player_d_dash\defense-dash-lt10_SeasonType_Regular_20Season_Season_2019-20.xlsx')
pd1_20 = pd.read_excel('player_d_dash\defense-dash-lt10_SeasonType_Regular_20Season_Season_2020-21.xlsx')
pd1_21 = pd.read_excel('player_d_dash\defense-dash-lt10_SeasonType_Regular_20Season_Season_2021-22.xlsx')

pdz1 = [pd1_17, pd1_18, pd1_19, pd1_20, pd1_21]

In [ ]:
pd1_20

In [ ]:
n = 17
for p in pdz1:
    p['within_10ft_defense_decile'] = pd.qcut(p['DFG%'], 10, labels = False)
    n += 1
    p.to_excel('aggregate_player_data\\withn_10ft_defense_'+ str(n)  + '.xlsx')

## Player Contracts

In [ ]:
sample_contract = pd.read_csv('player_contracts\Aaron Brooks.csv')

In [ ]:
namezy = sample_contract.columns[1]

In [ ]:
sample_contract.head(3)

In [ ]:
sample_contract = sample_contract[namezy].str.split(':', expand=True)
sample_contract.head(3)

In [ ]:
sample_contract.rename(columns = {0: 'Date', 1 : 'Transaction'}, inplace = True)
sample_contract.head(3)

In [ ]:
sample_contract['name'] = namezy

sample_contract

### Player Contracts - Automation

In [ ]:
path = 'C:\\Users\\tmcro\\OneDrive\\0_NBA_Statistitcs_2021\\NBA Current Studies\\Flatiron Phase 3 Project\\player_contracts'
p = os.listdir(path)
pf = pd.DataFrame(p)
pf

In [ ]:
path = 'C:\\Users\\tmcro\\OneDrive\\0_NBA_Statistitcs_2021\\NBA Current Studies\\Flatiron Phase 3 Project\\player_contracts_outputs'
p = os.listdir(path)
pf = pd.DataFrame(p)
pf

In [ ]:
cont = pd.read_excel('player_contracts\\Aaron Gordon.xlsx')
cont

In [ ]:
nmm = cont.columns[1]

In [ ]:
cont = cont[nmm].str.split(':', expand=True)

In [ ]:
cont.rename(columns = {0: 'Date', 1 : 'Transaction'}, inplace = True)
cont

In [ ]:
cont['Player'] = nmm

In [ ]:
cont.to_excel('player_contracts_outputs\\' + str(nmm) + '.xlsx')

In [ ]:
for n in pf[0]:
    contract = pd.read_excel('player_contracts/' + str(n))
    nm = contract.columns[1]
    contract = contract[nm].str.split(':', expand=True)
    contract.rename(columns = {0: 'Date', 1 : 'Transaction'}, inplace = True)
    contract['Player'] = nm
    contract.to_excel('player_contracts_outputs\\' + str(n))


In [ ]:
path = 'C:\\Users\\tmcro\\OneDrive\\0_NBA_Statistitcs_2021\\NBA Current Studies\\Flatiron Phase 3 Project\\player_contracts_outputs'
p = os.listdir(path)
pf = pd.DataFrame(p)
pf

In [ ]:
pf_reg = pf.loc[pf[0].astype(str).str.contains('.xlsx')]

In [ ]:
pf_reg

In [ ]:
# append the data
appended_contracts = []

for file in pf_reg[0]:
    data = pd.read_csv('player_contracts\\' + str(file))
    dat = pd.DataFrame(data)
    appended_contracts.append(dat)

appended_contracts = pd.concat(appended_contracts)
appended_contracts

#### Get Game IDs

In [10]:
path = Path("C:\\Users\\tmcro\\OneDrive\\0_NBA_Statistitcs_2021\\NBA Current Studies\\Data\\BigDataBall PbP Logs\\Combined Stats 2020 - 2021 PBP Logs")
p = os.listdir(path)
pf = pd.DataFrame(p)

In [12]:
# Str Split to New Columns
pf = pf[0].str.split('-', expand=True)

In [18]:
# Make hm_date_ids
pf['hm_date_id']= pf[4].astype(str).str[4:7]+ '-' + pf[2].astype(str).str[0:2] + '-' + pf[1].astype(str) + '-' + pf[0].astype(str).str[1:]
pf

,0,1,2,3,4,hm_date_id
0,2020,2021_combined,stats,Copy.csv,None,-st-2021_combined-020
1,2020,2021_combined,stats.csv,None,None,-st-2021_combined-020
2,[2020,12,22],0022000001,GSW@BKN.csv,BKN-22-12-2020
3,[2020,12,22],0022000002,LAC@LAL.csv,LAL-22-12-2020
4,[2020,12,23],0022000003,MIL@BOS.csv,BOS-23-12-2020
...,...,...,...,...,...,...
1124,[2021,06,02],0042000135,ATL@NYK.csv,NYK-02-06-2021
1125,[2021,06,02],0042000145,MEM@UTA.csv,UTA-02-06-2021
1126,[2021,06,02],0042000175,DAL@LAC.csv,LAC-02-06-2021
1127,[2021,06,03],0042000156,PHX@LAL.csv,LAL-03-06-2021


In [19]:
pf.to_excel('2020_game_ids.xlsx')

In [31]:
path = Path("C:\\Users\\tmcro\\OneDrive\\0_NBA_Statistitcs_2021\\NBA Current Studies\\Data\\BigDataBall PbP Logs\\2019-2020_NBA_PbP_Logs")
p = os.listdir(path)
pf = pd.DataFrame(p)

In [32]:
pf = pf[3:]

In [33]:
pf = pf[0].str.split('-', expand=True)
pf

,0,1,2,3,4
3,[2019,10,22],0021900001,NOP@TOR.csv
4,[2019,10,22],0021900002,LAL@LAC.csv
5,[2019,10,23],0021900003,CHI@CHA.csv
6,[2019,10,23],0021900004,DET@IND.csv
7,[2019,10,23],0021900005,CLE@ORL.csv
...,...,...,...,...,...
1141,[2020,10,02],0041900402,MIA@LAL.csv
1142,[2020,10,04],0041900403,LAL@MIA.csv
1143,[2020,10,06],0041900404,LAL@MIA.csv
1144,[2020,10,09],0041900405,MIA@LAL.csv


In [34]:
pf['hm_date_id']= pf[4].astype(str).str[4:7]+ '-' + pf[2].astype(str).str[0:2] + '-' + pf[1].astype(str) + '-' + pf[0].astype(str).str[1:]
pf

,0,1,2,3,4,hm_date_id
3,[2019,10,22],0021900001,NOP@TOR.csv,TOR-22-10-2019
4,[2019,10,22],0021900002,LAL@LAC.csv,LAC-22-10-2019
5,[2019,10,23],0021900003,CHI@CHA.csv,CHA-23-10-2019
6,[2019,10,23],0021900004,DET@IND.csv,IND-23-10-2019
7,[2019,10,23],0021900005,CLE@ORL.csv,ORL-23-10-2019
...,...,...,...,...,...,...
1141,[2020,10,02],0041900402,MIA@LAL.csv,LAL-02-10-2020
1142,[2020,10,04],0041900403,LAL@MIA.csv,MIA-04-10-2020
1143,[2020,10,06],0041900404,LAL@MIA.csv,MIA-06-10-2020
1144,[2020,10,09],0041900405,MIA@LAL.csv,LAL-09-10-2020


In [35]:
pf.to_excel('2019_game_ids.xlsx')

In [36]:
# Now get 2018
path = Path("C:\\Users\\tmcro\\OneDrive\\0_NBA_Statistitcs_2021\\NBA Current Studies\\Data\\BigDataBall PbP Logs\\2018-2019_NBA_PbP_Logs")
p = os.listdir(path)
pf = pd.DataFrame(p)

In [37]:
pf = pf[1:]
pf = pf[0].str.split('-', expand=True)
pf

,0,1,2,3,4
1,[2018,10,16],0021800001,PHI@BOS.csv
2,[2018,10,16],0021800002,OKC@GSW.csv
3,[2018,10,17],0021800003,MIL@CHA.csv
4,[2018,10,17],0021800004,BKN@DET.csv
5,[2018,10,17],0021800005,MEM@IND.csv
...,...,...,...,...,...
1308,[2019,06,02],0041800402,GSW@TOR.csv
1309,[2019,06,05],0041800403,TOR@GSW.csv
1310,[2019,06,07],0041800404,TOR@GSW.csv
1311,[2019,06,10],0041800405,GSW@TOR.csv


In [38]:
pf['hm_date_id']= pf[4].astype(str).str[4:7]+ '-' + pf[2].astype(str).str[0:2] + '-' + pf[1].astype(str) + '-' + pf[0].astype(str).str[1:]
pf

,0,1,2,3,4,hm_date_id
1,[2018,10,16],0021800001,PHI@BOS.csv,BOS-16-10-2018
2,[2018,10,16],0021800002,OKC@GSW.csv,GSW-16-10-2018
3,[2018,10,17],0021800003,MIL@CHA.csv,CHA-17-10-2018
4,[2018,10,17],0021800004,BKN@DET.csv,DET-17-10-2018
5,[2018,10,17],0021800005,MEM@IND.csv,IND-17-10-2018
...,...,...,...,...,...,...
1308,[2019,06,02],0041800402,GSW@TOR.csv,TOR-02-06-2019
1309,[2019,06,05],0041800403,TOR@GSW.csv,GSW-05-06-2019
1310,[2019,06,07],0041800404,TOR@GSW.csv,GSW-07-06-2019
1311,[2019,06,10],0041800405,GSW@TOR.csv,TOR-10-06-2019


In [39]:
pf.to_excel('2018_game_ids.xlsx')

In [41]:
path = Path("C:\\Users\\tmcro\\OneDrive\\0_NBA_Statistitcs_2021\\NBA Current Studies\\Data\\BigDataBall PbP Logs\\2017-2018_NBA_PbP_Logs")
p = os.listdir(path)
pf = pd.DataFrame(p)

In [42]:
pf = pf[1:]
pf = pf[0].str.split('-', expand=True)
pf

,0,1,2,3,4
1,[2017,10,17],0021700001,BOS@CLE.csv
2,[2017,10,17],0021700002,HOU@GSW.csv
3,[2017,10,18],0021700003,CHA@DET.csv
4,[2017,10,18],0021700004,BKN@IND.csv
5,[2017,10,18],0021700005,MIA@ORL.csv
...,...,...,...,...,...
1308,[2018,05,28],0041700317,GSW@HOU.csv
1309,[2018,05,31],0041700401,CLE@GSW.csv
1310,[2018,06,03],0041700402,CLE@GSW.csv
1311,[2018,06,06],0041700403,GSW@CLE.csv


In [43]:
pf['hm_date_id']= pf[4].astype(str).str[4:7]+ '-' + pf[2].astype(str).str[0:2] + '-' + pf[1].astype(str) + '-' + pf[0].astype(str).str[1:]
pf

,0,1,2,3,4,hm_date_id
1,[2017,10,17],0021700001,BOS@CLE.csv,CLE-17-10-2017
2,[2017,10,17],0021700002,HOU@GSW.csv,GSW-17-10-2017
3,[2017,10,18],0021700003,CHA@DET.csv,DET-18-10-2017
4,[2017,10,18],0021700004,BKN@IND.csv,IND-18-10-2017
5,[2017,10,18],0021700005,MIA@ORL.csv,ORL-18-10-2017
...,...,...,...,...,...,...
1308,[2018,05,28],0041700317,GSW@HOU.csv,HOU-28-05-2018
1309,[2018,05,31],0041700401,CLE@GSW.csv,GSW-31-05-2018
1310,[2018,06,03],0041700402,CLE@GSW.csv,GSW-03-06-2018
1311,[2018,06,06],0041700403,GSW@CLE.csv,CLE-06-06-2018


In [44]:
pf.to_excel('2017_game_ids.xlsx')

## Try to replicate what you did in excel
1) Create starters_by_game

In [45]:
#load 
big_2020 = pd.read_csv('C:\\Users\\tmcro\\OneDrive\\0_NBA_Statistitcs_2021\\NBA Current Studies\\Data\\BigDataBall PbP Logs\\Combined Stats 2020 - 2021 PBP Logs\\2020-2021_combined-stats - Copy.csv')
big_2020.head(3)

c:\Users\tmcro\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [55]:
first_play = big_2020.loc[big_2020['play_id']== 1]

Index(['game_id', 'data_set', 'date', 'a1', 'a2', 'a3', 'a4', 'a5', 'h1', 'h2',
       'h3', 'h4', 'h5', 'period', 'away_score', 'home_score',
       'remaining_time', 'elapsed', 'play_length', 'play_id', 'team',
       'event_type', 'assist', 'away', 'home', 'block', 'entered', 'left',
       'num', 'opponent', 'outof', 'player', 'points', 'possession', 'reason',
       'result', 'steal', 'type', 'shot_distance', 'original_x', 'original_y',
       'converted_x', 'converted_y', 'description'],
      dtype='object')

In [56]:
colz = ['game_id', 'data_set', 'date', 'a1', 'a2', 'a3', 'a4', 'a5', 'h1', 'h2',
       'h3', 'h4', 'h5']
first_play = first_play[colz]
first_play.head(3)

,game_id,data_set,date,a1,a2,a3,a4,a5,h1,h2,h3,h4,h5
0,22000001,2020-21 Regular Season,2020-12-22,Stephen Curry,James Wiseman,Andrew Wiggins,Kelly Oubre Jr.,Eric Paschall,DeAndre Jordan,Kyrie Irving,Kevin Durant,Joe Harris,Spencer Dinwiddie
517,22000002,2020-21 Regular Season,2020-12-22,Serge Ibaka,Paul George,Patrick Beverley,Nicolas Batum,Kawhi Leonard,Anthony Davis,LeBron James,Kentavious Caldwell-Pope,Dennis Schroder,Marc Gasol
1016,22000003,2020-21 Regular Season,2020-12-23,Brook Lopez,Giannis Antetokounmpo,Khris Middleton,Donte DiVincenzo,Jrue Holiday,Tristan Thompson,Jaylen Brown,Marcus Smart,Jayson Tatum,Daniel Theis
1488,22000004,2020-21 Regular Season,2020-12-23,Dwight Powell,Josh Richardson,Luka Doncic,Tim Hardaway Jr.,Dorian Finney-Smith,Deandre Ayton,Jae Crowder,Devin Booker,Mikal Bridges,Chris Paul
1963,22000010,2020-21 Regular Season,2020-12-23,Cody Zeller,Terry Rozier,Devonte' Graham,Gordon Hayward,P.J. Washington,Andre Drummond,Collin Sexton,Darius Garland,Larry Nance Jr.,Isaac Okoro
...,...,...,...,...,...,...,...,...,...,...,...,...,...
519121,42000135,2020-21 Playoffs,2021-06-02,Trae Young,Bogdan Bogdanovic,Clint Capela,John Collins,De'Andre Hunter,Taj Gibson,Julius Randle,Derrick Rose,RJ Barrett,Reggie Bullock
519572,42000145,2020-21 Playoffs,2021-06-02,Jonas Valanciunas,Ja Morant,Dillon Brooks,Kyle Anderson,Jaren Jackson Jr.,Rudy Gobert,Royce O'Neale,Donovan Mitchell,Bojan Bogdanovic,Mike Conley
520006,42000175,2020-21 Playoffs,2021-06-02,Kristaps Porzingis,Luka Doncic,Tim Hardaway Jr.,Dorian Finney-Smith,Boban Marjanovic,Marcus Morris Sr.,Reggie Jackson,Paul George,Nicolas Batum,Kawhi Leonard
520461,42000156,2020-21 Playoffs,2021-06-03,Deandre Ayton,Chris Paul,Jae Crowder,Devin Booker,Mikal Bridges,Marc Gasol,Dennis Schroder,Anthony Davis,Kentavious Caldwell-Pope,LeBron James


In [57]:
# Now we need the GAMEID and GAMES Info
games_2020 = pd.read_excel('2020_game_ids.xlsx')
games_2020

,Unnamed: 0,0,1,2,3,4,hm_date_id
0,0,2020,2021_combined,stats,Copy.csv,NaN,-st-2021_combined-020
1,1,2020,2021_combined,stats.csv,NaN,NaN,-st-2021_combined-020
2,2,[2020,12,22],0022000001,GSW@BKN.csv,BKN-22-12-2020
3,3,[2020,12,22],0022000002,LAC@LAL.csv,LAL-22-12-2020
4,4,[2020,12,23],0022000003,MIL@BOS.csv,BOS-23-12-2020
...,...,...,...,...,...,...,...
1124,1124,[2021,06,02],0042000135,ATL@NYK.csv,NYK-02-06-2021
1125,1125,[2021,06,02],0042000145,MEM@UTA.csv,UTA-02-06-2021
1126,1126,[2021,06,02],0042000175,DAL@LAC.csv,LAC-02-06-2021
1127,1127,[2021,06,03],0042000156,PHX@LAL.csv,LAL-03-06-2021


In [58]:
games_2020 = games_2020[2:]
games_2020 = games_2020.drop(columns = ['Unnamed: 0', 0, 1, 2])
games_2020

,3,4,hm_date_id
2,0022000001,GSW@BKN.csv,BKN-22-12-2020
3,0022000002,LAC@LAL.csv,LAL-22-12-2020
4,0022000003,MIL@BOS.csv,BOS-23-12-2020
5,0022000004,DAL@PHX.csv,PHX-23-12-2020
6,0022000010,CHA@CLE.csv,CLE-23-12-2020
...,...,...,...
1124,0042000135,ATL@NYK.csv,NYK-02-06-2021
1125,0042000145,MEM@UTA.csv,UTA-02-06-2021
1126,0042000175,DAL@LAC.csv,LAC-02-06-2021
1127,0042000156,PHX@LAL.csv,LAL-03-06-2021


In [62]:
games_2020 = games_2020.rename(columns = {3: 'game_id', 4: 'matchup'})
games_2020.head(3)

,game_id,matchup,hm_date_id
2,0022000001,GSW@BKN.csv,BKN-22-12-2020
3,0022000002,LAC@LAL.csv,LAL-22-12-2020
4,0022000003,MIL@BOS.csv,BOS-23-12-2020


In [67]:
games_2020['game_id'] = games_2020['game_id'].astype(np.int64)
games_2020.head(3)

,game_id,matchup,hm_date_id
2,22000001,GSW@BKN.csv,BKN-22-12-2020
3,22000002,LAC@LAL.csv,LAL-22-12-2020
4,22000003,MIL@BOS.csv,BOS-23-12-2020


In [68]:
starters_2020 = pd.merge(first_play, games_2020, on= 'game_id', how = 'left')
starters_2020

,game_id,data_set,date,a1,a2,a3,a4,a5,h1,h2,h3,h4,h5,matchup,hm_date_id
0,22000001,2020-21 Regular Season,2020-12-22,Stephen Curry,James Wiseman,Andrew Wiggins,Kelly Oubre Jr.,Eric Paschall,DeAndre Jordan,Kyrie Irving,Kevin Durant,Joe Harris,Spencer Dinwiddie,GSW@BKN.csv,BKN-22-12-2020
1,22000002,2020-21 Regular Season,2020-12-22,Serge Ibaka,Paul George,Patrick Beverley,Nicolas Batum,Kawhi Leonard,Anthony Davis,LeBron James,Kentavious Caldwell-Pope,Dennis Schroder,Marc Gasol,LAC@LAL.csv,LAL-22-12-2020
2,22000003,2020-21 Regular Season,2020-12-23,Brook Lopez,Giannis Antetokounmpo,Khris Middleton,Donte DiVincenzo,Jrue Holiday,Tristan Thompson,Jaylen Brown,Marcus Smart,Jayson Tatum,Daniel Theis,MIL@BOS.csv,BOS-23-12-2020
3,22000004,2020-21 Regular Season,2020-12-23,Dwight Powell,Josh Richardson,Luka Doncic,Tim Hardaway Jr.,Dorian Finney-Smith,Deandre Ayton,Jae Crowder,Devin Booker,Mikal Bridges,Chris Paul,DAL@PHX.csv,PHX-23-12-2020
4,22000010,2020-21 Regular Season,2020-12-23,Cody Zeller,Terry Rozier,Devonte' Graham,Gordon Hayward,P.J. Washington,Andre Drummond,Collin Sexton,Darius Garland,Larry Nance Jr.,Isaac Okoro,CHA@CLE.csv,CLE-23-12-2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,42000135,2020-21 Playoffs,2021-06-02,Trae Young,Bogdan Bogdanovic,Clint Capela,John Collins,De'Andre Hunter,Taj Gibson,Julius Randle,Derrick Rose,RJ Barrett,Reggie Bullock,ATL@NYK.csv,NYK-02-06-2021
1123,42000145,2020-21 Playoffs,2021-06-02,Jonas Valanciunas,Ja Morant,Dillon Brooks,Kyle Anderson,Jaren Jackson Jr.,Rudy Gobert,Royce O'Neale,Donovan Mitchell,Bojan Bogdanovic,Mike Conley,MEM@UTA.csv,UTA-02-06-2021
1124,42000175,2020-21 Playoffs,2021-06-02,Kristaps Porzingis,Luka Doncic,Tim Hardaway Jr.,Dorian Finney-Smith,Boban Marjanovic,Marcus Morris Sr.,Reggie Jackson,Paul George,Nicolas Batum,Kawhi Leonard,DAL@LAC.csv,LAC-02-06-2021
1125,42000156,2020-21 Playoffs,2021-06-03,Deandre Ayton,Chris Paul,Jae Crowder,Devin Booker,Mikal Bridges,Marc Gasol,Dennis Schroder,Anthony Davis,Kentavious Caldwell-Pope,LeBron James,PHX@LAL.csv,LAL-03-06-2021


In [69]:
starters_2020.to_excel('starters_by_game/2020_2021_starters.xlsx')

#### 2019

In [76]:
#load 
big_2019 = pd.read_csv('C:\\Users\\tmcro\\OneDrive\\0_NBA_Statistitcs_2021\\NBA Current Studies\\Data\\BigDataBall PbP Logs\\2019-2020_NBA_PbP_Logs\\[10-22-2019]-[10-11-2020]-combined-stats.csv')
big_2019.head(3)

c:\Users\tmcro\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,game_id,data_set,date,a1,a2,a3,a4,a5,h1,h2,...,reason,result,steal,type,shot_distance,original_x,original_y,converted_x,converted_y,description
0,"=""0021900001""",2019-2020 Regular Season,2019-10-22,Jrue Holiday,Brandon Ingram,Derrick Favors,JJ Redick,Lonzo Ball,OG Anunoby,Pascal Siakam,...,NaN,NaN,NaN,start of period,NaN,NaN,NaN,NaN,NaN,NaN
1,"=""0021900001""",2019-2020 Regular Season,2019-10-22,Jrue Holiday,Brandon Ingram,Derrick Favors,JJ Redick,Lonzo Ball,OG Anunoby,Pascal Siakam,...,NaN,NaN,NaN,jump ball,NaN,NaN,NaN,NaN,NaN,Jump Ball Gasol vs. Favors: Tip to Ball
2,"=""0021900001""",2019-2020 Regular Season,2019-10-22,Jrue Holiday,Brandon Ingram,Derrick Favors,JJ Redick,Lonzo Ball,OG Anunoby,Pascal Siakam,...,NaN,missed,NaN,unknown,11.0,2,114,24.8,16.4,MISS Ball 11' Driving Floating Jump Shot


In [81]:
# Get a list of files in a folder
path = 'C:\\Users\\tmcro\\OneDrive\\0_NBA_Statistitcs_2021\\NBA Current Studies\\Data\\BigDataBall PbP Logs\\2019-2020_NBA_PbP_Logs'
p = os.listdir(path)
pf = pd.DataFrame(p)



In [82]:
pf

,0
0,(10-22-2019)-(10-11-2020)-combined-stats.csv
1,(10-22-2019)-(10-11-2020)-combined-stats.xlsx
2,[10-22-2019]-[10-11-2020]-combined-stats.csv
3,[2019-10-22]-0021900001-NOP@TOR.csv
4,[2019-10-22]-0021900002-LAL@LAC.csv
...,...
1141,[2020-10-02]-0041900402-MIA@LAL.csv
1142,[2020-10-04]-0041900403-LAL@MIA.csv
1143,[2020-10-06]-0041900404-LAL@MIA.csv
1144,[2020-10-09]-0041900405-MIA@LAL.csv


In [84]:
# Get just the ones you want based on file names
pf_reg = pf[3:]

# append the data
appended_data = []

for file in pf_reg[0]:
    data = pd.read_csv(path +'\\' + file)
    appended_data.append(data)

appended_data = pd.concat(appended_data)
appended_data

,game_id,data_set,date,a1,a2,a3,a4,a5,h1,h2,...,reason,result,steal,type,shot_distance,original_x,original_y,converted_x,converted_y,description
0,"=""0021900001""",2019-2020 Regular Season,2019-10-22,Jrue Holiday,Brandon Ingram,Derrick Favors,JJ Redick,Lonzo Ball,OG Anunoby,Pascal Siakam,...,NaN,NaN,NaN,start of period,NaN,NaN,NaN,NaN,NaN,NaN
1,"=""0021900001""",2019-2020 Regular Season,2019-10-22,Jrue Holiday,Brandon Ingram,Derrick Favors,JJ Redick,Lonzo Ball,OG Anunoby,Pascal Siakam,...,NaN,NaN,NaN,jump ball,NaN,NaN,NaN,NaN,NaN,Jump Ball Gasol vs. Favors: Tip to Ball
2,"=""0021900001""",2019-2020 Regular Season,2019-10-22,Jrue Holiday,Brandon Ingram,Derrick Favors,JJ Redick,Lonzo Ball,OG Anunoby,Pascal Siakam,...,NaN,missed,NaN,unknown,11.0,2,114,24.8,16.4,MISS Ball 11' Driving Floating Jump Shot
3,"=""0021900001""",2019-2020 Regular Season,2019-10-22,Jrue Holiday,Brandon Ingram,Derrick Favors,JJ Redick,Lonzo Ball,OG Anunoby,Pascal Siakam,...,NaN,NaN,NaN,rebound offensive,NaN,NaN,NaN,NaN,NaN,Favors REBOUND (Off:1 Def:0)
4,"=""0021900001""",2019-2020 Regular Season,2019-10-22,Jrue Holiday,Brandon Ingram,Derrick Favors,JJ Redick,Lonzo Ball,OG Anunoby,Pascal Siakam,...,NaN,made,NaN,unknown,1.0,0,-6,25.0,4.4,Favors 1' Tip Layup Shot (2 PTS)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,'0041900406,2019-20 Playoffs,2020-10-11,Quinn Cook,Jared Dudley,Dwight Howard,Alex Caruso,Kentavious Caldwell-Pope,Derrick Jones Jr.,Kelly Olynyk,...,NaN,NaN,NaN,rebound offensive,NaN,NaN,NaN,NaN,NaN,Olynyk REBOUND (Off:3 Def:4)
441,'0041900406,2019-20 Playoffs,2020-10-11,Quinn Cook,Jared Dudley,Dwight Howard,Alex Caruso,Kentavious Caldwell-Pope,Derrick Jones Jr.,Kelly Olynyk,...,NaN,made,NaN,Putback Layup,2.0,13,21,26.3,86.9,Olynyk 2' Putback Layup (9 PTS)
442,'0041900406,2019-20 Playoffs,2020-10-11,Quinn Cook,Jared Dudley,Dwight Howard,Alex Caruso,Kentavious Caldwell-Pope,Derrick Jones Jr.,Kelly Olynyk,...,NaN,made,NaN,3PT Jump Shot,30.0,-17,299,26.7,34.9,Howard 30' 3PT Jump Shot (3 PTS) (Caruso 5 AST)
443,'0041900406,2019-20 Playoffs,2020-10-11,Quinn Cook,Jared Dudley,Dwight Howard,Alex Caruso,Kentavious Caldwell-Pope,Derrick Jones Jr.,Kelly Olynyk,...,NaN,made,NaN,3PT Jump Shot,29.0,193,221,44.3,66.9,S. Hill 29' 3PT Jump Shot (5 PTS)


In [127]:
appended_data.to_excel('2019_starters.xlsx')

In [128]:
big_2019 = pd.read_excel('2019_starters.xlsx')
big_2019

,Unnamed: 0,game_id,game_id.1,data_set,date,a1,a2,a3,a4,a5,...,reason,result,steal,type,shot_distance,original_x,original_y,converted_x,converted_y,description
0,0,21900001,21900001,2019-2020 Regular Season,2019-10-22,Jrue Holiday,Brandon Ingram,Derrick Favors,JJ Redick,Lonzo Ball,...,NaN,NaN,NaN,start of period,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0021900001,21900001,2019-2020 Regular Season,2019-10-22,Jrue Holiday,Brandon Ingram,Derrick Favors,JJ Redick,Lonzo Ball,...,NaN,NaN,NaN,jump ball,NaN,NaN,NaN,NaN,NaN,Jump Ball Gasol vs. Favors: Tip to Ball
2,2,0021900001,21900001,2019-2020 Regular Season,2019-10-22,Jrue Holiday,Brandon Ingram,Derrick Favors,JJ Redick,Lonzo Ball,...,NaN,missed,NaN,unknown,11.0,2,114,24.8,16.4,MISS Ball 11' Driving Floating Jump Shot
3,3,0021900001,21900001,2019-2020 Regular Season,2019-10-22,Jrue Holiday,Brandon Ingram,Derrick Favors,JJ Redick,Lonzo Ball,...,NaN,NaN,NaN,rebound offensive,NaN,NaN,NaN,NaN,NaN,Favors REBOUND (Off:1 Def:0)
4,4,0021900001,21900001,2019-2020 Regular Season,2019-10-22,Jrue Holiday,Brandon Ingram,Derrick Favors,JJ Redick,Lonzo Ball,...,NaN,made,NaN,unknown,1.0,0,-6,25.0,4.4,Favors 1' Tip Layup Shot (2 PTS)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543144,440,'0041900406,41900406,2019-20 Playoffs,2020-10-11,Quinn Cook,Jared Dudley,Dwight Howard,Alex Caruso,Kentavious Caldwell-Pope,...,NaN,NaN,NaN,rebound offensive,NaN,NaN,NaN,NaN,NaN,Olynyk REBOUND (Off:3 Def:4)
543145,441,'0041900406,41900406,2019-20 Playoffs,2020-10-11,Quinn Cook,Jared Dudley,Dwight Howard,Alex Caruso,Kentavious Caldwell-Pope,...,NaN,made,NaN,Putback Layup,2.0,13,21,26.3,86.9,Olynyk 2' Putback Layup (9 PTS)
543146,442,'0041900406,41900406,2019-20 Playoffs,2020-10-11,Quinn Cook,Jared Dudley,Dwight Howard,Alex Caruso,Kentavious Caldwell-Pope,...,NaN,made,NaN,3PT Jump Shot,30.0,-17,299,26.7,34.9,Howard 30' 3PT Jump Shot (3 PTS) (Caruso 5 AST)
543147,443,'0041900406,41900406,2019-20 Playoffs,2020-10-11,Quinn Cook,Jared Dudley,Dwight Howard,Alex Caruso,Kentavious Caldwell-Pope,...,NaN,made,NaN,3PT Jump Shot,29.0,193,221,44.3,66.9,S. Hill 29' 3PT Jump Shot (5 PTS)


In [78]:
big_2019.columns

Index(['game_id', 'data_set', 'date', 'a1', 'a2', 'a3', 'a4', 'a5', 'h1', 'h2',
       'h3', 'h4', 'h5', 'period', 'away_score', 'home_score',
       'remaining_time', 'elapsed', 'play_length', 'play_id', 'team',
       'event_type', 'assist', 'away', 'home', 'block', 'entered', 'left',
       'num', 'opponent', 'outof', 'player', 'points', 'possession', 'reason',
       'result', 'steal', 'type', 'shot_distance', 'original_x', 'original_y',
       'converted_x', 'converted_y', 'description'],
      dtype='object')

In [132]:
first_play = big_2019.loc[big_2019['play_id']== 2]
first_play

,Unnamed: 0,game_id,game_id.1,data_set,date,a1,a2,a3,a4,a5,...,reason,result,steal,type,shot_distance,original_x,original_y,converted_x,converted_y,description
0,0,21900001,21900001,2019-2020 Regular Season,2019-10-22,Jrue Holiday,Brandon Ingram,Derrick Favors,JJ Redick,Lonzo Ball,...,NaN,NaN,NaN,start of period,NaN,NaN,NaN,NaN,NaN,NaN
573,0,0021900002,21900002,2019-2020 Regular Season,2019-10-22,Anthony Davis,LeBron James,JaVale McGee,Danny Green,Avery Bradley,...,NaN,NaN,NaN,start of period,NaN,NaN,NaN,NaN,NaN,NaN
1039,0,21900003,21900003,2019-2020 Regular Season,10/23/2019,Otto Porter Jr.,Lauri Markkanen,Wendell Carter Jr.,Zach LaVine,Tomas Satoransky,...,NaN,NaN,NaN,start of period,NaN,NaN,NaN,NaN,NaN,NaN
1536,0,0021900004,21900004,2019-2020 Regular Season,2019-10-23,Tony Snell,Markieff Morris,Andre Drummond,Bruce Brown,Reggie Jackson,...,NaN,NaN,NaN,start of period,NaN,NaN,NaN,NaN,NaN,NaN
2025,0,21900005,21900005,2019-2020 Regular Season,10/23/2019,Cedi Osman,Kevin Love,Tristan Thompson,Darius Garland,Collin Sexton,...,NaN,NaN,NaN,start of period,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540937,0,'0041900402,41900402,2019-20 Playoffs,2020-10-02,Meyers Leonard,Jimmy Butler,Duncan Robinson,Jae Crowder,Tyler Herro,...,NaN,NaN,NaN,start of period,NaN,NaN,NaN,NaN,NaN,NaN
541383,0,'0041900403,41900403,2019-20 Playoffs,2020-10-04,Dwight Howard,LeBron James,Anthony Davis,Danny Green,Kentavious Caldwell-Pope,...,NaN,NaN,NaN,start of period,NaN,NaN,NaN,NaN,NaN,NaN
541835,0,'0041900404,41900404,2019-20 Playoffs,2020-10-06,Dwight Howard,LeBron James,Anthony Davis,Kentavious Caldwell-Pope,Danny Green,...,NaN,NaN,NaN,start of period,NaN,NaN,NaN,NaN,NaN,NaN
542264,0,'0041900405,41900405,2019-20 Playoffs,2020-10-09,Bam Adebayo,Tyler Herro,Jimmy Butler,Duncan Robinson,Jae Crowder,...,NaN,NaN,NaN,start of period,NaN,NaN,NaN,NaN,NaN,NaN


In [133]:
colz = ['game_id','game_id.1'	, 'data_set', 'date', 'a1', 'a2', 'a3', 'a4', 'a5', 'h1', 'h2',
       'h3', 'h4', 'h5']
first_play = first_play[colz]
first_play.head(3)

,game_id,game_id.1,data_set,date,a1,a2,a3,a4,a5,h1,h2,h3,h4,h5
0,21900001,21900001,2019-2020 Regular Season,2019-10-22,Jrue Holiday,Brandon Ingram,Derrick Favors,JJ Redick,Lonzo Ball,OG Anunoby,Pascal Siakam,Marc Gasol,Kyle Lowry,Fred VanVleet
573,0021900002,21900002,2019-2020 Regular Season,2019-10-22,Anthony Davis,LeBron James,JaVale McGee,Danny Green,Avery Bradley,Kawhi Leonard,Patrick Patterson,Ivica Zubac,Landry Shamet,Patrick Beverley
1039,21900003,21900003,2019-2020 Regular Season,10/23/2019,Otto Porter Jr.,Lauri Markkanen,Wendell Carter Jr.,Zach LaVine,Tomas Satoransky,Miles Bridges,P.J. Washington,Cody Zeller,Dwayne Bacon,Terry Rozier


In [134]:
# Now we need the GAMEID and GAMES Info
games_2019 = pd.read_excel('2019_game_ids.xlsx')
games_2019

,Unnamed: 0,0,1,2,3,4,hm_date_id
0,3,[2019,10,22],21900001,NOP@TOR.csv,TOR-22-10-2019
1,4,[2019,10,22],21900002,LAL@LAC.csv,LAC-22-10-2019
2,5,[2019,10,23],21900003,CHI@CHA.csv,CHA-23-10-2019
3,6,[2019,10,23],21900004,DET@IND.csv,IND-23-10-2019
4,7,[2019,10,23],21900005,CLE@ORL.csv,ORL-23-10-2019
...,...,...,...,...,...,...,...
1138,1141,[2020,10,02],41900402,MIA@LAL.csv,LAL-02-10-2020
1139,1142,[2020,10,04],41900403,LAL@MIA.csv,MIA-04-10-2020
1140,1143,[2020,10,06],41900404,LAL@MIA.csv,MIA-06-10-2020
1141,1144,[2020,10,09],41900405,MIA@LAL.csv,LAL-09-10-2020


In [135]:

games_2019 = games_2019.drop(columns = ['Unnamed: 0', 0, 1, 2])
games_2019 = games_2019.rename(columns = {3: 'game_id', 4: 'matchup'})
games_2019.head(3)

,game_id,matchup,hm_date_id
0,21900001,NOP@TOR.csv,TOR-22-10-2019
1,21900002,LAL@LAC.csv,LAC-22-10-2019
2,21900003,CHI@CHA.csv,CHA-23-10-2019


In [136]:
games_2019['game_id'] = games_2019['game_id'].astype(np.int64)
games_2019.head(3)

,game_id,matchup,hm_date_id
0,21900001,NOP@TOR.csv,TOR-22-10-2019
1,21900002,LAL@LAC.csv,LAC-22-10-2019
2,21900003,CHI@CHA.csv,CHA-23-10-2019


In [138]:
starters_2019 = pd.merge(first_play, games_2019, left_on = 'game_id.1', right_on = 'game_id', how = 'left')
starters_2019

,game_id_x,game_id.1,data_set,date,a1,a2,a3,a4,a5,h1,h2,h3,h4,h5,game_id_y,matchup,hm_date_id
0,21900001,21900001,2019-2020 Regular Season,2019-10-22,Jrue Holiday,Brandon Ingram,Derrick Favors,JJ Redick,Lonzo Ball,OG Anunoby,Pascal Siakam,Marc Gasol,Kyle Lowry,Fred VanVleet,21900001,NOP@TOR.csv,TOR-22-10-2019
1,0021900002,21900002,2019-2020 Regular Season,2019-10-22,Anthony Davis,LeBron James,JaVale McGee,Danny Green,Avery Bradley,Kawhi Leonard,Patrick Patterson,Ivica Zubac,Landry Shamet,Patrick Beverley,21900002,LAL@LAC.csv,LAC-22-10-2019
2,21900003,21900003,2019-2020 Regular Season,10/23/2019,Otto Porter Jr.,Lauri Markkanen,Wendell Carter Jr.,Zach LaVine,Tomas Satoransky,Miles Bridges,P.J. Washington,Cody Zeller,Dwayne Bacon,Terry Rozier,21900003,CHI@CHA.csv,CHA-23-10-2019
3,0021900004,21900004,2019-2020 Regular Season,2019-10-23,Tony Snell,Markieff Morris,Andre Drummond,Bruce Brown,Reggie Jackson,T.J. Warren,Domantas Sabonis,Myles Turner,Jeremy Lamb,Malcolm Brogdon,21900004,DET@IND.csv,IND-23-10-2019
4,21900005,21900005,2019-2020 Regular Season,10/23/2019,Cedi Osman,Kevin Love,Tristan Thompson,Darius Garland,Collin Sexton,Jonathan Isaac,Aaron Gordon,Nikola Vucevic,Evan Fournier,D.J. Augustin,21900005,CLE@ORL.csv,ORL-23-10-2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1137,'0041900402,41900402,2019-20 Playoffs,2020-10-02,Meyers Leonard,Jimmy Butler,Duncan Robinson,Jae Crowder,Tyler Herro,Dwight Howard,LeBron James,Anthony Davis,Kentavious Caldwell-Pope,Danny Green,41900402,MIA@LAL.csv,LAL-02-10-2020
1138,'0041900403,41900403,2019-20 Playoffs,2020-10-04,Dwight Howard,LeBron James,Anthony Davis,Danny Green,Kentavious Caldwell-Pope,Meyers Leonard,Tyler Herro,Duncan Robinson,Jimmy Butler,Jae Crowder,41900403,LAL@MIA.csv,MIA-04-10-2020
1139,'0041900404,41900404,2019-20 Playoffs,2020-10-06,Dwight Howard,LeBron James,Anthony Davis,Kentavious Caldwell-Pope,Danny Green,Bam Adebayo,Tyler Herro,Duncan Robinson,Jae Crowder,Jimmy Butler,41900404,LAL@MIA.csv,MIA-06-10-2020
1140,'0041900405,41900405,2019-20 Playoffs,2020-10-09,Bam Adebayo,Tyler Herro,Jimmy Butler,Duncan Robinson,Jae Crowder,Dwight Howard,LeBron James,Kentavious Caldwell-Pope,Danny Green,Anthony Davis,41900405,MIA@LAL.csv,LAL-09-10-2020


In [139]:
starters_2019.to_excel('starters_by_game/2019_2020_starters.xlsx')

### 2018

In [23]:
path = 'C:\\Users\\tmcro\\OneDrive\\0_NBA_Statistitcs_2021\\NBA Current Studies\\Data\\BigDataBall PbP Logs\\2017-2018_NBA_PbP_Logs'
p = os.listdir(path)
pf = pd.DataFrame(p)


In [24]:
pf

,0
0,[10-17-2017]-[06-08-2018]-combined-stats.csv
1,[2017-10-17]-0021700001-BOS@CLE.csv
2,[2017-10-17]-0021700002-HOU@GSW.csv
3,[2017-10-18]-0021700003-CHA@DET.csv
4,[2017-10-18]-0021700004-BKN@IND.csv
...,...
1308,[2018-05-28]-0041700317-GSW@HOU.csv
1309,[2018-05-31]-0041700401-CLE@GSW.csv
1310,[2018-06-03]-0041700402-CLE@GSW.csv
1311,[2018-06-06]-0041700403-GSW@CLE.csv


In [ ]:
'BigDataBall PbP Logs\\2018-2019_NBA_PbP_Logs\\[10-17-2017]-[06-08-2018]-combined-stats.csv'

In [25]:
#load 
big_2017 = pd.read_csv('C:\\Users\\tmcro\\OneDrive\\0_NBA_Statistitcs_2021\\NBA Current Studies\\Data\\BigDataBall PbP Logs\\2017-2018_NBA_PbP_Logs\\[10-17-2017]-[06-08-2018]-combined-stats.csv',encoding = 'unicode_escape')



c:\Users\tmcro\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (0,38,39,40,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [26]:
big_2017.head(3)

,game_id,data_set,date,a1,a2,a3,a4,a5,h1,h2,...,reason,result,steal,type,shot_distance,original_x,original_y,converted_x,converted_y,description
0,21700001,2017-2018 Regular Season,10/17/2017,Gordon Hayward,Jayson Tatum,Al Horford,Jaylen Brown,Kyrie Irving,LeBron James,Jae Crowder,...,NaN,NaN,NaN,start of period,NaN,NaN,NaN,NaN,NaN,NaN
1,21700001,2017-2018 Regular Season,10/17/2017,Gordon Hayward,Jayson Tatum,Al Horford,Jaylen Brown,Kyrie Irving,LeBron James,Jae Crowder,...,NaN,NaN,NaN,jump ball,NaN,NaN,NaN,NaN,NaN,Jump Ball Love vs. Horford: Tip to Irving
2,21700001,2017-2018 Regular Season,10/17/2017,Gordon Hayward,Jayson Tatum,Al Horford,Jaylen Brown,Kyrie Irving,LeBron James,Jae Crowder,...,NaN,made,NaN,unknown,10,-1,100,25.1,15,Irving 10' Driving Floating Jump Shot (2 PTS) ...


In [27]:
first_play = big_2017.loc[big_2017['play_id' ]== 2]

In [28]:
colz = ['game_id', 'data_set', 'date', 'a1', 'a2', 'a3', 'a4', 'a5', 'h1', 'h2',
       'h3', 'h4', 'h5']
first_play = first_play[colz]
first_play.head(3)

,game_id,data_set,date,a1,a2,a3,a4,a5,h1,h2,h3,h4,h5
0,21700001,2017-2018 Regular Season,10/17/2017,Gordon Hayward,Jayson Tatum,Al Horford,Jaylen Brown,Kyrie Irving,LeBron James,Jae Crowder,Kevin Love,Dwyane Wade,Derrick Rose
477,21700002,2017-2018 Regular Season,10/17/2017,Trevor Ariza,Ryan Anderson,Clint Capela,James Harden,Chris Paul,Kevin Durant,Draymond Green,Zaza Pachulia,Klay Thompson,Stephen Curry
934,21700003,2017-2018 Regular Season,10/18/2017,Marvin Williams,Dwayne Bacon,Dwight Howard,Jeremy Lamb,Kemba Walker,Stanley Johnson,Tobias Harris,Andre Drummond,Avery Bradley,Reggie Jackson


In [4]:
# Now we need the GAMEID and GAMES Info
games_2017 = pd.read_excel('2017_game_ids.xlsx')
games_2017

,Unnamed: 0,0,1,2,3,4,hm_date_id
0,1,[2017,10,17],21700001,BOS@CLE.csv,CLE-17-10-2017
1,2,[2017,10,17],21700002,HOU@GSW.csv,GSW-17-10-2017
2,3,[2017,10,18],21700003,CHA@DET.csv,DET-18-10-2017
3,4,[2017,10,18],21700004,BKN@IND.csv,IND-18-10-2017
4,5,[2017,10,18],21700005,MIA@ORL.csv,ORL-18-10-2017
...,...,...,...,...,...,...,...
1307,1308,[2018,5,28],41700317,GSW@HOU.csv,HOU-28-05-2018
1308,1309,[2018,5,31],41700401,CLE@GSW.csv,GSW-31-05-2018
1309,1310,[2018,6,03],41700402,CLE@GSW.csv,GSW-03-06-2018
1310,1311,[2018,6,06],41700403,GSW@CLE.csv,CLE-06-06-2018


In [1]:
games_2017

NameError: name 'games_2017' is not defined

In [30]:

games_2017 = games_2017.drop(columns = ['Unnamed: 0', 0, 1, 2])
games_2017

,3,4,hm_date_id
0,21700001,BOS@CLE.csv,CLE-17-10-2017
1,21700002,HOU@GSW.csv,GSW-17-10-2017
2,21700003,CHA@DET.csv,DET-18-10-2017
3,21700004,BKN@IND.csv,IND-18-10-2017
4,21700005,MIA@ORL.csv,ORL-18-10-2017
...,...,...,...
1307,41700317,GSW@HOU.csv,HOU-28-05-2018
1308,41700401,CLE@GSW.csv,GSW-31-05-2018
1309,41700402,CLE@GSW.csv,GSW-03-06-2018
1310,41700403,GSW@CLE.csv,CLE-06-06-2018


In [31]:
games_2017 = games_2017.rename(columns = {3: 'game_id', 4: 'matchup'})
games_2017.head(3)

,game_id,matchup,hm_date_id
0,21700001,BOS@CLE.csv,CLE-17-10-2017
1,21700002,HOU@GSW.csv,GSW-17-10-2017
2,21700003,CHA@DET.csv,DET-18-10-2017


In [32]:
games_2017['game_id'] = games_2017['game_id'].astype(np.int64)
games_2017.head(3)

,game_id,matchup,hm_date_id
0,21700001,BOS@CLE.csv,CLE-17-10-2017
1,21700002,HOU@GSW.csv,GSW-17-10-2017
2,21700003,CHA@DET.csv,DET-18-10-2017


In [33]:
starters_2017 = pd.merge(first_play, games_2017, on= 'game_id', how = 'left')
starters_2017

,game_id,data_set,date,a1,a2,a3,a4,a5,h1,h2,h3,h4,h5,matchup,hm_date_id
0,21700001,2017-2018 Regular Season,10/17/2017,Gordon Hayward,Jayson Tatum,Al Horford,Jaylen Brown,Kyrie Irving,LeBron James,Jae Crowder,Kevin Love,Dwyane Wade,Derrick Rose,BOS@CLE.csv,CLE-17-10-2017
1,21700002,2017-2018 Regular Season,10/17/2017,Trevor Ariza,Ryan Anderson,Clint Capela,James Harden,Chris Paul,Kevin Durant,Draymond Green,Zaza Pachulia,Klay Thompson,Stephen Curry,HOU@GSW.csv,GSW-17-10-2017
2,21700003,2017-2018 Regular Season,10/18/2017,Marvin Williams,Dwayne Bacon,Dwight Howard,Jeremy Lamb,Kemba Walker,Stanley Johnson,Tobias Harris,Andre Drummond,Avery Bradley,Reggie Jackson,CHA@DET.csv,DET-18-10-2017
3,21700004,2017-2018 Regular Season,10/18/2017,Rondae Hollis-Jefferson,DeMarre Carroll,Timofey Mozgov,D'Angelo Russell,Jeremy Lin,Bojan Bogdanovic,Thaddeus Young,Myles Turner,Victor Oladipo,Darren Collison,BKN@IND.csv,IND-18-10-2017
4,21700005,2017-2018 Regular Season,10/18/2017,Josh Richardson,Kelly Olynyk,Hassan Whiteside,Dion Waiters,Goran Dragic,Evan Fournier,Aaron Gordon,Nikola Vucevic,Terrence Ross,Elfrid Payton,MIA@ORL.csv,ORL-18-10-2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307,41700317,2018 Playoff,5/28/2018,Kevin Durant,Draymond Green,Kevon Looney,Klay Thompson,Stephen Curry,Trevor Ariza,PJ Tucker,Clint Capela,Eric Gordon,James Harden,GSW@HOU.csv,HOU-28-05-2018
1308,41700401,2018 Playoff,5/31/2018,LeBron James,Kevin Love,Tristan Thompson,JR Smith,George Hill,Kevin Durant,Draymond Green,Kevon Looney,Klay Thompson,Stephen Curry,CLE@GSW.csv,GSW-31-05-2018
1309,41700402,2018 Playoff,6/3/2018,LeBron James,Kevin Love,Tristan Thompson,JR Smith,George Hill,Kevin Durant,Draymond Green,JaVale McGee,Klay Thompson,Stephen Curry,CLE@GSW.csv,GSW-03-06-2018
1310,41700403,2018 Playoff,6/6/2018,Kevin Durant,Draymond Green,JaVale McGee,Klay Thompson,Stephen Curry,LeBron James,Kevin Love,Tristan Thompson,JR Smith,George Hill,GSW@CLE.csv,CLE-06-06-2018


In [22]:
starters_2017.to_excel('starters_by_game/2017_2018_starters.xlsx')